学习AlphaFold,OpenFold,ESMFold

In [1]:
import torch
import torch.nn as nn
from matplotlib import pyplot as plt 
import numpy as np

In [84]:
# 函数1: 学习torch.split,在指定维度将tensor分成许多chunk tensor组成的tuple
# 参数二可以写为一个single chunk的大小，要么是许多chunk size的大小加起来等于10
x = torch.arange(10)
x_chunk = torch.split(x,5,dim=0)
# E.g.
    #    # [*, N_res, H * P_q * 3]
    #     q_pts = self.linear_q_points(s)

    #     # This is kind of clunky, but it's how the original does it
    #     # [*, N_res, H * P_q, 3]
    #     q_pts = torch.split(q_pts, q_pts.shape[-1] // 3, dim=-1)
    #     q_pts = torch.stack(q_pts, dim=-1)
N_res = 100
H = 10
P_q = 4
P_v = 8
q_pts = torch.randn(5,N_res,H*P_q*3)
q_pts = torch.split(q_pts, q_pts.shape[-1] // 3, dim=-1)
# q_pts = torch.stack(q_pts, dim=-1)

In [16]:
# 函数 1.5 : torch.chunk(input,chunks,dim=0)
# chunks指的是将tensor变成几个chunk
# chunks 是分成的几个块，chunk size是每一个块内部的大小

In [ ]:
# 函数2 : torch.unbind(input, dim=0)
# Removes a tensor dimension.
# Returns a tuple of all slices along a given dimension, already without it.
# torch.unbind : 把某一个维度按1进行切割
# torch.split  : 把某一个维度按照指定的chunk size进行切割
torch.unbind(torch.arange(10),dim=0) == torch.split(torch.arange(10),1,dim=0)
# E.g. *torch.unbind(o_pt, dim=-1) 其中*表示解压unbind产生的tuple 当作实参数
o_pt = torch.randn((N_res,H*P_v,3))
print(*torch.unbind(o_pt,dim=-1))

In [83]:
# 函数3 : torch.nn.Softplut(beta=1,threshold=20)
# SoftPlus is a smooth approximation to the ReLU function and can be used to constrain the output of a machine to always be positive.
a = np.linspace(-10.0,10.0,num=1000)


In [4]:
H=16
head_weights = torch.randn((H,))

In [5]:
head_weights.view(*())

tensor([ 0.6482, -0.9007, -0.7718, -0.1868, -0.4537,  0.7549, -0.1214, -0.6609,
         2.3571, -0.3103, -2.0868,  0.8933, -1.5520, -0.1901,  0.5534, -1.4012])

In [38]:
# class 内部的装饰器 1.
# 一共有三种class method的类型: instance method; statistic method; class method
# instance method绑定的对象是instance，必须先创建一个对象之后才能调用该方法(函数定义时形参是self)
# static method创建的时候不需要添加self 形参,可以在不创建对象的时候就调用该函数，避免了创建对象所带来的内存消耗;另一个角度理解是static method本身就是一个普通的函数，只是挂靠在class的这个命名空间底下而言，不需要实例化对象
# classmethod  类方法主要是为了优雅的创建多个对象
class MyDog:
    def __init__(self) -> None:
        pass
    def instance_method(self): # self 形参接受的是实例化后对象的IP地址
        print("My dog is an instance method")
    
    @staticmethod
    def static_method():
        print("My dog is a static method")
    
    @classmethod
    def class_method(cls): # cls参数表示这个类本身
        print(f"My dog is a class method,given {cls}")


My dog is a class method,given <class '__main__.MyDog'>


In [55]:
# class 内部的装饰器 2.
# @property 将一个method的调用方法变成属性的调用方法
# @property 将某个方法变成只读属性
# @xxx.setter 把该属性进行赋值
# 注意属性的方法名不要和属性名重名,否则self.birth调用时会递归的找self.birth这个函数造成栈溢出
class Student:
    def __init__(self) -> None:
        self._birth = 10
    @property
    def birth(self):
        return self._birth

    @birth.setter
    def birth(self,value):
        self._birth = value
a = Student()
a.birth = 100

### Module 0. Rigid Transformation

如果对坐标做变换，如何存储transformation

1. Rotation 是一个类似张量的 Rotation 对象，每一个point代表一个Rotation
2. 

In [88]:
from typing import Optional,Tuple
# Q1 : 3*3 旋转矩阵到旋转四元数的变换？3*3旋转正交矩阵自由度不是只有3?
# identity rotation matrix eye(3)
def identity_rot_mats(
    batch_dims: Tuple[int], 
    dtype: Optional[torch.dtype] = None, 
    device: Optional[torch.device] = None, 
    requires_grad: bool = True,
) -> torch.Tensor:
    rots = torch.eye(
        3, dtype=dtype, device=device, requires_grad=requires_grad
    )
    rots = rots.view(*((1,) * len(batch_dims)), 3, 3)
    rots = rots.expand(*batch_dims, -1, -1)
    rots = rots.contiguous()

    return rots

# identity translation vector [0,0,0]
def identity_trans(
    batch_dims: Tuple[int], 
    dtype: Optional[torch.dtype] = None,
    device: Optional[torch.device] = None, 
    requires_grad: bool = True,
) -> torch.Tensor:
    trans = torch.zeros(
        (*batch_dims, 3), 
        dtype=dtype, 
        device=device, 
        requires_grad=requires_grad
    )
    return trans

# identity quaternion [1, 0, 0, 0]
def identity_quats(
    batch_dims: Tuple[int], 
    dtype: Optional[torch.dtype] = None,
    device: Optional[torch.device] = None, 
    requires_grad: bool = True,
) -> torch.Tensor:
    quat = torch.zeros(
        (*batch_dims, 4), 
        dtype=dtype, 
        device=device, 
        requires_grad=requires_grad
    )

    with torch.no_grad():
        # quat [* , 4] 
        quat[..., 0] = 1

    return quat
_quat_elements = ["a", "b", "c", "d"]
_qtr_keys = [l1 + l2 for l1 in _quat_elements for l2 in _quat_elements]
_qtr_ind_dict = {key: ind for ind, key in enumerate(_qtr_keys)}


### Evoformer .Triangular multiplicative update

Algorithm 11 : Triangular multiplicative update using "outgoing" edges, for ij : ik , jk

Algorithm 112 : Triangular multiplicative update using "incoming" edges, for ij : ki,kj

### Evoformer .Triangular self attention

Algorithm 13 : Triangular gated self-attention around starting node. (row-wise)

Algorithm 112 : Triangular gated self-attention around ending node. (column-wise)

In [ ]:
class TriangleAttention(nn.Module):
    def __init__(
        self, c_in, c_hidden, no_heads, starting=True, inf=1e9
    ):
        """
        Args:
            c_in:
                Input channel dimension
            c_hidden:
                Overall hidden channel dimension (not per-head)
            no_heads:
                Number of attention heads
        """
        super(TriangleAttention, self).__init__()

        self.c_in = c_in
        self.c_hidden = c_hidden
        self.no_heads = no_heads
        self.starting = starting
        self.inf = inf

        self.layer_norm = LayerNorm(self.c_in)

        self.linear = Linear(c_in, self.no_heads, bias=False, init="normal")

        self.mha = Attention(
            self.c_in, self.c_in, self.c_in, self.c_hidden, self.no_heads
        )

    @torch.jit.ignore
    def _chunk(self,
        x: torch.Tensor,
        biases: List[torch.Tensor],
        chunk_size: int,
        use_memory_efficient_kernel: bool = False,
        use_lma: bool = False,
        inplace_safe: bool = False,
    ) -> torch.Tensor:
        "triangle! triangle!"
        mha_inputs = {
            "q_x": x,
            "kv_x": x,
            "biases": biases,
        }

        return chunk_layer(
            partial(
                self.mha, 
                use_memory_efficient_kernel=use_memory_efficient_kernel,
                use_lma=use_lma
            ),
            mha_inputs,
            chunk_size=chunk_size,
            no_batch_dims=len(x.shape[:-2]),
            _out=x if inplace_safe else None,
        )

    def forward(self, 
        x: torch.Tensor, 
        mask: Optional[torch.Tensor] = None,
        chunk_size: Optional[int] = None,
        use_memory_efficient_kernel: bool = False,
        use_lma: bool = False,
        inplace_safe: bool = False,
    ) -> torch.Tensor:
        """
        Args:
            x:
                [*, I, J, C_in] input tensor (e.g. the pair representation)
        Returns:
            [*, I, J, C_in] output tensor
        """ 
        if mask is None:
            # [*, I, J]
            mask = x.new_ones(
                x.shape[:-1],
            )

        if(not self.starting):
            x = x.transpose(-2, -3)
            mask = mask.transpose(-1, -2)

        # [*, I, J, C_in]
        x = self.layer_norm(x)

        # [*, I, 1, 1, J]
        mask_bias = (self.inf * (mask - 1))[..., :, None, None, :]

        # [*, H, I, J]
        triangle_bias = permute_final_dims(self.linear(x), (2, 0, 1))

        # [*, 1, H, I, J]
        triangle_bias = triangle_bias.unsqueeze(-4)

        biases = [mask_bias, triangle_bias]

        if chunk_size is not None:
            x = self._chunk(
                x, 
                biases, 
                chunk_size, 
                use_memory_efficient_kernel=use_memory_efficient_kernel,
                use_lma=use_lma,
                inplace_safe=inplace_safe,
            )
        else:
            x = self.mha(
                q_x=x, 
                kv_x=x, 
                biases=biases, 
                use_memory_efficient_kernel=use_memory_efficient_kernel,
                use_lma=use_lma
            )

        if(not self.starting):
            x = x.transpose(-2, -3)

        return x


# Implements Algorithm 13
TriangleAttentionStartingNode = TriangleAttention


class TriangleAttentionEndingNode(TriangleAttention):
    """
    Implements Algorithm 14.
    """
    __init__ = partialmethod(TriangleAttention.__init__, starting=False)

### Structure Module 1. IPA
PS : 
1. H最开始肯定放在N_res后面,但是在attention score中一定是放在N_res前面，也就是H,N_res,N_res
2. key,value是二维的情况下怎么画图


总结：
- 两套qkv,全部从single chain representation中得到
  - qkv 为标准的多头注意力, 将c_s 投影为H * C_hidden
  - q_pts, k_pts, v_pts 显示的构建了坐标的注意力机制,将c_s 投影为H*P_qk/P_v * 3,之后直接用Rigid transforamtion变换这些点点坐标
- 三个attention score (affinities) 和一个bias
  - pair-bias 由pair-representation直接将c_z投影到H的维度进行加和
  - dot-product 由标准的多头注意力机制qkv计算得来
  - squared distance affinities 由点注意力机制构建 $o_{pt} = -\frac{\gamma ^h W_c}{2}\sum\limits_{p}||q^{hp}_{i}-k^{hp}_{j} ||^2$
    - 具体计算步骤就是输入一个[*, N_res, H, P_q, 3]的query q_pts和key k_pts
    - pt_att = q_pts.unsqueeze(-4) - k_pts.unsqueeze(-5) query key氨基酸每个原子中的xyz坐标之差，利用brodcast来进行对称
    - pt_att = pt_att** 2得到xyz距离平方
    - pt_att = torch.sum(pt_att,dim=-1) 求和xyz坐标，得到每个residue的每个原子的坐标距离
      - 源代码里是这样写的 sum(*torch.unbind(pt_att,dim=-1)),似乎没什么差别
    - 加入weight
    - 加入squared mask,常规操作:
      - mask [*, N_res]
      - -inf * (mask.unsqueeze(-1) * mask.unsqueeze(-2)).unsqueeze(-3) -> [*, 1, N_res, N_res]
- 四个output value, 最后concat h,q
  - pair-representation value
    - [ *, H, N_res, N_res].transpose(-2,-3) @ [ *, N_res, N_res, c_z] -> [ *, N_res, H, c_z] -> [ *, N_res, H * c_z]
  - dot-product value
    - [ *, H, N_res, N_res] matmaul [ \*, H, N_res, C_hidden] -> [ \*, H, N_res ,C_hidden] -> [ *, N_res, H * C_hidden]
  - o_pt (point attention value)
    - input
      - v_pts : [ *, N_res, H, P_v, 3] 
      - a     : [ *, H, N_res, N_res]
    - output : o_pt : [ *, N_res, H, P_v, 3], split into 3 points : [ *, N_res, H * P_v] * 3
  - o_pt_norm
    - v_pts : $\sqrt{v_{pts}^2}$
- 1个输出
  - 更新之后的s

In [ ]:
# From OpenFold
class InvariantPointAttention(nn.Module):
    """
    Implements Algorithm 22.
    """
    def __init__(
        self,
        c_s: int,
        c_z: int,
        c_hidden: int,
        no_heads: int,
        no_qk_points: int,
        no_v_points: int,
        inf: float = 1e5,
        eps: float = 1e-8,
    ):
        """
        Args:
            c_s:
                Single representation channel dimension
            c_z:
                Pair representation channel dimension
            c_hidden:
                Hidden channel dimension
            no_heads:
                Number of attention heads
            no_qk_points:
                Number of query/key points to generate
            no_v_points:
                Number of value points to generate
        """
        super(InvariantPointAttention, self).__init__()

        self.c_s = c_s
        self.c_z = c_z
        self.c_hidden = c_hidden
        self.no_heads = no_heads
        self.no_qk_points = no_qk_points
        self.no_v_points = no_v_points
        self.inf = inf
        self.eps = eps

        # These linear layers differ from their specifications in the
        # supplement. There, they lack bias and use Glorot initialization.
        # Here as in the official source, they have bias and use the default
        # Lecun initialization.
        hc = self.c_hidden * self.no_heads
        self.linear_q = Linear(self.c_s, hc)
        self.linear_kv = Linear(self.c_s, 2 * hc) # 2*hc 表示一个是key,一个是value

        hpq = self.no_heads * self.no_qk_points * 3 # 表示point attention 中 query 需要投影到的维度
        self.linear_q_points = Linear(self.c_s, hpq)

        hpkv = self.no_heads * (self.no_qk_points + self.no_v_points) * 3 # 表示point attention 中 key + value 需要投影到的维度
        self.linear_kv_points = Linear(self.c_s, hpkv)

        hpv = self.no_heads * self.no_v_points * 3

        self.linear_b = Linear(self.c_z, self.no_heads)

        self.head_weights = nn.Parameter(torch.zeros((no_heads))) # head weights 表示一个可学习的gamma h参数用来控制point attention affinity中的不同head
        ipa_point_weights_init_(self.head_weights)

        concat_out_dim = self.no_heads * (
            self.c_z + self.c_hidden + self.no_v_points * 4
        ) # 最终拼接时的dimension, c_z表示pair representation的output,  c_hidden 表示从single representaion计算的output, 4个no_v_points表示3个坐标(此处concat p)+1个norm；外侧乘H表示concat (H)
        self.linear_out = Linear(concat_out_dim, self.c_s, init="final")

        self.softmax = nn.Softmax(dim=-1)
        self.softplus = nn.Softplus()

    def forward(
        self,
        s: torch.Tensor,
        z: Optional[torch.Tensor],
        r: Rigid,
        mask: torch.Tensor,
        inplace_safe: bool = False,
        _offload_inference: bool = False,
        _z_reference_list: Optional[Sequence[torch.Tensor]] = None,
    ) -> torch.Tensor:
        """
        Args:
            s:
                [*, N_res, C_s] single representation
            z:
                [*, N_res, N_res, C_z] pair representation
            r:
                [*, N_res] transformation object
            mask:
                [*, N_res] mask
        Returns:
            [*, N_res, C_s] single representation update
        """
        if(_offload_inference and inplace_safe):
            z = _z_reference_list
        else:
            z = [z]
       
        #######################################
        # Generate scalar and point activations
        #######################################
        # [*, N_res, H * C_hidden]
        q = self.linear_q(s)
        kv = self.linear_kv(s)

        # [*, N_res, H, C_hidden]
        q = q.view(q.shape[:-1] + (self.no_heads, -1))

        # [*, N_res, H, 2 * C_hidden]
        kv = kv.view(kv.shape[:-1] + (self.no_heads, -1))

        # [*, N_res, H, C_hidden]
        k, v = torch.split(kv, self.c_hidden, dim=-1)

        # [*, N_res, H * P_q * 3]
        q_pts = self.linear_q_points(s)

        # This is kind of clunky, but it's how the original does it
        # 注意Rigid instance 做slicing的时候是对*做的(batch dimension)，而不用考虑最后的3*3
        q_pts = torch.split(q_pts, q_pts.shape[-1] // 3, dim=-1)
        # [*, N_res, H * P_q, 3]
        q_pts = torch.stack(q_pts, dim=-1)
        q_pts = r[..., None].apply(q_pts)

        # [*, N_res, H, P_q, 3]
        q_pts = q_pts.view(
            q_pts.shape[:-2] + (self.no_heads, self.no_qk_points, 3)
        )

        # [*, N_res, H * (P_q + P_v) * 3]
        kv_pts = self.linear_kv_points(s)

        # [*, N_res, H * (P_q + P_v), 3]
        kv_pts = torch.split(kv_pts, kv_pts.shape[-1] // 3, dim=-1)
        kv_pts = torch.stack(kv_pts, dim=-1)
        # r[...,None] -> 
        # Rigid : _rot       = [ *, N_res, 1, 3, 3]  
        #         _trans     = [ *, N_res, 1, 3]
        kv_pts = r[..., None].apply(kv_pts)


        # [*, N_res, H, (P_q + P_v), 3]
        kv_pts = kv_pts.view(kv_pts.shape[:-2] + (self.no_heads, -1, 3))

        # [*, N_res, H, P_q/P_v, 3]
        k_pts, v_pts = torch.split(
            kv_pts, [self.no_qk_points, self.no_v_points], dim=-2
        )

        ##########################
        # Compute attention scores
        ##########################
        # [*, N_res, N_res, H]
        b = self.linear_b(z[0])
        
        if(_offload_inference):
            assert(sys.getrefcount(z[0]) == 2)
            z[0] = z[0].cpu()

        # [*, H, N_res, N_res]
        if(is_fp16_enabled()):
            with torch.cuda.amp.autocast(enabled=False):
                a = torch.matmul(
                    permute_final_dims(q.float(), (1, 0, 2)),  # [*, H, N_res, C_hidden]
                    permute_final_dims(k.float(), (1, 2, 0)),  # [*, H, C_hidden, N_res]
                )
        else:
            # q,k,v original [*, N_res, H, C_hidden]
            a = torch.matmul(
                permute_final_dims(q, (1, 0, 2)),  # [*, H, N_res, C_hidden]
                permute_final_dims(k, (1, 2, 0)),  # [*, H, C_hidden, N_res]
            ) # attention affinity [*, H, N_res, N_res] 
        
        a *= math.sqrt(1.0 / (3 * self.c_hidden))
        a += (math.sqrt(1.0 / 3) * permute_final_dims(b, (2, 0, 1)))
        # bias from  [*, N_res, N_res, H] -> [*, H, N_res, N_res]

        # [*, N_res, N_res, H, P_q, 3]
        # Original q_pts,k_pts,v_pts after global transformation  -> [*, N_res, H, P_qk, 3]
        # broadcast from [*, N_res, 1, H, P_q, 3] - [*, N_res, 1, H, P_q, 3]
        pt_att = q_pts.unsqueeze(-4) - k_pts.unsqueeze(-5)
        if(inplace_safe):
            pt_att *= pt_att
        else:
            pt_att = pt_att ** 2

        # [*, N_res, N_res, H, P_q]
        pt_att = sum(torch.unbind(pt_att, dim=-1)) # 等价于 torch.sum(pt_att,dim=-1), 求两两点之间的膜长
        head_weights = self.softplus(self.head_weights).view(
            *((1,) * len(pt_att.shape[:-2]) + (-1, 1))
        ) # [*, 1, 1, H, 1]
        head_weights = head_weights * math.sqrt(
            1.0 / (3 * (self.no_qk_points * 9.0 / 2))
        )
        if(inplace_safe):
            pt_att *= head_weights
        else:
            pt_att = pt_att * head_weights

        # [*, N_res, N_res, H]
        pt_att = torch.sum(pt_att, dim=-1) * (-0.5)
        # [*, N_res, N_res]
        # Broadcast from [*, N_res, 1] * [*, 1, N_res]
        square_mask = mask.unsqueeze(-1) * mask.unsqueeze(-2)
        square_mask = self.inf * (square_mask - 1)

        # [*, H, N_res, N_res]
        pt_att = permute_final_dims(pt_att, (2, 0, 1))
        
        if(inplace_safe):
            a += pt_att
            del pt_att
            a += square_mask.unsqueeze(-3)
            # in-place softmax
            attn_core_inplace_cuda.forward_(
                a,
                reduce(mul, a.shape[:-1]),
                a.shape[-1],
            )
        else:
            # a from dot-product affinities and bias , will be added point attention 
            a = a + pt_att 
            a = a + square_mask.unsqueeze(-3)
            a = self.softmax(a)

        ################
        # Compute output
        ################
        # [*, N_res, H, C_hidden]
        o = torch.matmul(
            a, v.transpose(-2, -3).to(dtype=a.dtype)
        ).transpose(-2, -3)

        # [*, N_res, H * C_hidden]
        o = flatten_final_dims(o, 2)

        # [*, H, 3, N_res, P_v] 
        if(inplace_safe):
            v_pts = permute_final_dims(v_pts, (1, 3, 0, 2))
            o_pt = [
                torch.matmul(a, v.to(a.dtype)) 
                for v in torch.unbind(v_pts, dim=-3)
            ]
            o_pt = torch.stack(o_pt, dim=-3)
        else:
            o_pt = torch.sum(
                (
                    a[..., None, :, :, None]
                    * permute_final_dims(v_pts, (1, 3, 0, 2))[..., None, :, :]
                ),
                dim=-2,
            )
            # step by step
            # a[...,None,:,:,None] from [*, H, N_res, N_res]                -> [*, H, 1, N_res, N_res,    1]
            # v_pts from [*, N_res, H, P_v, 3] ->  [*, H, 3, N_res, P_v]    -> [*, H, 3, 1,     N_res,  P_v] 
            # 与 [*, H, N_res, N_res] @ [*, H, N_res, 3*P_v] 然后再把最后一个维度展开有啥区别?

        # [*, N_res, H, P_v, 3]
        o_pt = permute_final_dims(o_pt, (2, 0, 3, 1))
        # _rot : [*, N_res, 1, 1, 3, 3], _trans : [*, N_res, 1, 1, 3]
        o_pt = r[..., None, None].invert_apply(o_pt)

        # [*, N_res, H * P_v]
        o_pt_norm = flatten_final_dims(
            torch.sqrt(torch.sum(o_pt ** 2, dim=-1) + self.eps), 2
        )

        # [*, N_res, H * P_v, 3]
        o_pt = o_pt.reshape(*o_pt.shape[:-3], -1, 3)

        if(_offload_inference):
            z[0] = z[0].to(o_pt.device)

        # [*, N_res, H, C_z]
        # [*, N_res, H, N_res] @ [*, N_res, N_res, c_z] -> [*, N_res, H, c_z]
        o_pair = torch.matmul(a.transpose(-2, -3), z[0].to(dtype=a.dtype))

        # [*, N_res, H * C_z]
        o_pair = flatten_final_dims(o_pair, 2)

        # [*, N_res, C_s]
        s = self.linear_out(
            torch.cat(
                (o, *torch.unbind(o_pt, dim=-1), o_pt_norm, o_pair), dim=-1
            ).to(dtype=z[0].dtype)
            # concat list : H * (4*P_v + C_z + c_hidden)
            # o : [*, N_res, H*c_hidden]
            # * torch.unbind(o_pt, dim=-1),解压每一个坐标的维度 
            # - o_pt_p1 [*, N_res, H*P_v]
            # - o_pt_p2 [*, N_res, H*P_v]
            # - o_pt_p3 [*, N_res, H*P_v]
            # o_pt_norm : [*, N_res, H*P_v]
            # o_pair    : [*, N_res, H*C_z]
        )
        
        return s

## 2023.6.26 学习openfold chunk size相关函数

chunk size是每一小块里面的氨基酸数目，具体的for 循环次数是$\frac{N_{res}}{N_{chunk\ size}}$

#### 函数1：递归得到一个tensor dict的所有维度

In [9]:
def _fetch_dims(tree):
    shapes = []
    tree_type = type(tree)
    if tree_type is dict:
        for v in tree.values():
            shapes.extend(_fetch_dims(v))
    elif tree_type is list or tree_type is tuple:
        for t in tree:
            shapes.extend(_fetch_dims(t))
    elif tree_type is torch.Tensor:
        shapes.append(tree.shape)
    else:
        print(tree_type)
        raise ValueError("Not supported")

    return shapes

In [11]:
import torch.nn as nn
import torch
x = {"p1":torch.randn(1, 2, 3), "p2":torch.randn(1, 2, 3)}

In [12]:
_fetch_dims(x)

[torch.Size([1, 2, 3]), torch.Size([1, 2, 3])]

#### 将一个平铺之后的索引flat_idx转为在dims中的多维度

In [14]:
from typing import Tuple
def _flat_idx_to_idx(
    flat_idx: int,
    dims: Tuple[int],
) -> Tuple[int]:
    idx = []
    for d in reversed(dims):
        idx.append(flat_idx % d)
        flat_idx = flat_idx // d

    return tuple(reversed(idx))


In [15]:
array = [[0, 1, 2],
         [3, 4, 5]]
flat_idx = 4
dims = (2, 3)

result = _flat_idx_to_idx(flat_idx, dims)
print(result)


(1, 1)


#### Numpy/torch einsum function

Take openfold code as example

In [85]:
# 1. covariance matrix and outer product
N_seq = 77
N_res = 256
c = 128
msa_repre = torch.randn(N_seq,N_res,c)

i = 10
j = 20

ai = msa_repre[:,i,:] # [N_seq,c]
bj = msa_repre[:,j,:] # [N_seq,c]

In [4]:
covaraince = 1/N_seq * ai.T @ bj # covariance = 1/N \sum_{i=1}^{N} ais @ bis
outer_product = ai.unsqueeze(-1) @ bj.unsqueeze(1) # save all the rank1 matr
(covaraince - outer_product.mean(dim=0)) < 1e-5

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [88]:
# 2. torch.einsum is much faster than for loop to calculate the N^2 covariance matrix
a = nn.Linear(c,c)(msa_repre)
b = nn.Linear(c,c)(msa_repre)
a = a.transpose(-2,-3)
b = b.transpose(-2,-3)
print(a.shape,b.shape)

torch.Size([256, 77, 128]) torch.Size([256, 77, 128])


In [92]:
outer_einsum_zb = torch.einsum("...abc,dbf->adcf",a,b)
outer_einsum_zb /= N_seq

In [97]:
print(f"a shape {a.shape}") # N_res sets of input faetures with N_seq,c
outer_esinsum = torch.einsum("...bac,...dae->...bdce", a, b)
outer_esinsum = outer_esinsum / N_seq
print(f"outer shape {outer_esinsum.shape}") # N_res sets of outer product with N_seq,c,c

print(f"zb outer einsum is equal to outer product : {torch.allclose(outer_einsum_zb,outer_esinsum)}")

a shape torch.Size([256, 77, 128])
outer shape torch.Size([256, 256, 128, 128])
zb outer einsum is equal to outer product : True


In [11]:
outer = torch.zeros(N_res,N_res,c,c)
for i in range(N_res):
    for j in range(N_res):
        outer[i,j] = 1/N_seq * a[i,...].T @ b[j,...]
print(f"outer shape {outer.shape}") # N_res sets of outer product with N_seq,c,c
print(torch.all((outer_esinsum - outer).abs() < 1e-5))

outer shape torch.Size([256, 256, 128, 128])
tensor(True)


In [ ]:
# 3. how to understand einsum : Einstenin summation convention 
import numpy as np


In [15]:
# 3.1 Implicit summation: In implicit mode, the chosen subscripts are important since the axes of the output are reordered alphabetically
# Whenever a label is repeated it is summed,
x = np.array([1,2,3,4])
np.einsum("i,i",x,x)

#  If a label appears only once, it is not summed
np.einsum("i",x)


array([1, 2, 3, 4])

In [34]:
# 3.2 Explicit mode : In explicit mode the output can be directly controlled by specifying output subscript labels. This requires the identifier ‘->’ as well as the list of output subscript labels. This feature increases the flexibility of the function since summing can be disabled or forced when required. 
# same label will be summed multiplication, different label will be multiplication once

A = torch.randn(2)
B = torch.randn(2)
C_inner_product = torch.einsum("i,i->",A,B) # same label will be \sum A_i * B_i
C_outer_product = torch.einsum("i,j->ij",A,B) # different label will be A_i * B_j 
C_elementwise_product = torch.einsum("i,i->i",A,B) # different label will be A_i * B_i no sum , the axis will be preserved
print(f"inner product {C_inner_product.shape} outer product {C_outer_product.shape}")

A = torch.randn(3,3)
B = torch.randn(3,3)
C_matrix_multiplication = torch.einsum("ij,jk->ik",A,B) # same label will be \sum_{j=1}^N A_ij * B_jk
C_elementwise_multiplication = torch.einsum("ij,ij->ij",A,B) # different label will be A_ij * B_ij no sum , the axis will be preserved
print(f"matrix multiplication {C_matrix_multiplication.shape} elementwise multiplication {C_elementwise_multiplication.shape}")

inner product torch.Size([]) outer product torch.Size([2, 2])
matrix multiplication torch.Size([3, 3]) elementwise multiplication torch.Size([3, 3])


In [48]:
# element-wise multiplication and summation 
A = torch.randn(3,3)
B = torch.randn(3,3)
C = torch.einsum("ij,ij->",A,B) 
print(C,torch.sum(A*B))

tensor(-1.8008) tensor(-1.8008)


In [53]:
# kroneck product
A = torch.randn(3,3)
B = torch.randn(3,3)
C = torch.einsum("ij,mn->ijmn",A,B) 
print(C.shape)
# what is C
# C[i,j,m,n] = A[i,j] * B[m,n]
for i in range(3):
    for j in range(3):
        print(C[i,j] == A[i,j] * B)

torch.Size([3, 3, 3, 3])
tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])
tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])
tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])
tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])
tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])
tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])
tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])
tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])
tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])


In [56]:
# transpose product 1: A^T @ B
A = torch.randn(3,3)
B = torch.randn(3,3)
C = torch.einsum("ij,im->jm",A,B) 
print(C,"\n",A.T@B)

tensor([[ 4.9452,  3.6882, -1.3189],
        [-0.4733, -0.9803,  0.8564],
        [ 0.2147, -0.7913,  0.2626]]) 
 tensor([[ 4.9452,  3.6882, -1.3189],
        [-0.4733, -0.9803,  0.8564],
        [ 0.2147, -0.7913,  0.2626]])


In [57]:
# transpose product 2: A @ B^T
A = torch.randn(3,3)
B = torch.randn(3,3)
C = torch.einsum("ij,bj->ib",A,B) 
print(C,"\n",A@B.T)

tensor([[-0.6810,  1.1732,  0.0499],
        [ 0.8210, -0.6290, -0.1657],
        [-0.2496,  1.0134, -0.4312]]) 
 tensor([[-0.6810,  1.1732,  0.0499],
        [ 0.8210, -0.6290, -0.1657],
        [-0.2496,  1.0134, -0.4312]])


In [58]:
# practice
a = torch.randn(3)
b = torch.randn(3)
c = torch.einsum("i,i->",a,b)
d = torch.einsum("i,i->i",a,b)
e = torch.einsum("i,j->ij",a,b)

In [65]:
# trace and diagonal
A = torch.randn(3,3)
trace_A = torch.einsum("ii->",A) # summation Aii 
diag_A = torch.einsum("ii->i",A)

In [66]:
# sum on an axis
i,j,m,n = 3,4,5,6
A = torch.randn(i,j,m,n)
sum_A = torch.einsum("ijmn->ijm",A) # sum all the elements
sum_A_n = torch.sum(A,dim=-1)
print(torch.all(sum_A == sum_A_n))

tensor(True)


In [78]:
a = np.arange(60.).reshape(3,4,5) # i j k
b = np.arange(24.).reshape(4,3,2) # j i l
np.einsum('ijk,jil->kl', a, b)

array([[4400., 4730.],
       [4532., 4874.],
       [4664., 5018.],
       [4796., 5162.],
       [4928., 5306.]])

In [83]:
a = np.arange(60.).reshape(3,4,5) # i j k
b = np.arange(24.).reshape(4,3,2) # j i l
np.einsum('ijk,jil->kl', a, b) # kl is output subscript, ij is summation subscript, Okl = \sum_i \sum_j a_{ijk} * b_{jil}

tensor_contraction = np.zeros((5,2))
for i in range(5):
    for j in range(2):
        tensor_contraction[i,j] = np.trace(a[...,i] @ b[...,j])

In [84]:
tensor_contraction

array([[4400., 4730.],
       [4532., 4874.],
       [4664., 5018.],
       [4796., 5162.],
       [4928., 5306.]])

In [28]:
# practice alphafold outer product mean
outer_zb = torch.einsum("...abc,...dbe->...adce",a,b)

In [30]:
outer_zb = outer_zb / N_seq

In [ ]:
import torch
import torch.nn as nn
class OuterProductMean(nn.Module):
    """
    Implements Algorithm 10.
    """

    def __init__(self, c_m, c_z, c_hidden, eps=1e-3):
        """
        Args:
            c_m:
                MSA embedding channel dimension
            c_z:
                Pair embedding channel dimension
            c_hidden:
                Hidden channel dimension
        """
        super(OuterProductMean, self).__init__()

        self.c_m = c_m
        self.c_z = c_z
        self.c_hidden = c_hidden
        self.eps = eps

        self.layer_norm = nn.LayerNorm(c_m)
        self.linear_1 = nn.Linear(c_m, c_hidden)
        self.linear_2 = nn.Linear(c_m, c_hidden)
        self.linear_out = nn.Linear(c_hidden ** 2, c_z)

    def _opm(self, a, b):
        # [*, N_res, N_res, C, C]
        # a : [*, N_res, S, C]
        # b : [*, N_res, S, C]
        outer = torch.einsum("...bac,...dae->...bdce", a, b)

        # [*, N_res, N_res, C * C]
        outer = outer.reshape(outer.shape[:-2] + (-1,))

        # [*, N_res, N_res, C_z]
        outer = self.linear_out(outer)

        return outer
    
    def forward_by_zb(self,m):
        ## by zb to understand einsum, outer product mean to extract co-evolution information, chunk size 
        ln = self.layer_norm(m)
        a = self.linear_1(ln)
        b = self.linear_2(ln)
        return self._opm(a,b)

    @torch.jit.ignore
    def _chunk(self, 
        a: torch.Tensor, 
        b: torch.Tensor, 
        chunk_size: int
    ) -> torch.Tensor:
        # Since the "batch dim" in this case is not a true batch dimension
        # (in that the shape of the output depends on it), we need to
        # iterate over it ourselves
        # a.shape [N_seq, N_res, C] -> (-1,N_seq,N_res,C)
        a_reshape = a.reshape((-1,) + a.shape[-3:])
        b_reshape = b.reshape((-1,) + b.shape[-3:])
        out = []
        for a_prime, b_prime in zip(a_reshape, b_reshape):
            outer = chunk_layer(
                partial(self._opm, b=b_prime),
                {"a": a_prime},
                chunk_size=chunk_size,
                no_batch_dims=1,
            )
            out.append(outer)

        # For some cursed reason making this distinction saves memory
        if(len(out) == 1):
            outer = out[0].unsqueeze(0)
        else:
            outer = torch.stack(out, dim=0)

        outer = outer.reshape(a.shape[:-3] + outer.shape[1:])

        return outer

    def _forward(self, 
        m: torch.Tensor, 
        mask: Optional[torch.Tensor] = None,
        chunk_size: Optional[int] = None,
        inplace_safe: bool = False,
    ) -> torch.Tensor:
        """
        Args:
            m:
                [*, N_seq, N_res, C_m] MSA embedding
            mask:
                [*, N_seq, N_res] MSA mask
        Returns:
            [*, N_res, N_res, C_z] pair embedding update
        """
        if mask is None:
            mask = m.new_ones(m.shape[:-1])

        # [*, N_seq, N_res, C_m]
        ln = self.layer_norm(m)

        # [*, N_seq, N_res, C]
        mask = mask.unsqueeze(-1)
        a = self.linear_1(ln) 
        a = a * mask
        
        b = self.linear_2(ln) 
        b = b * mask

        del ln

        a = a.transpose(-2, -3)
        b = b.transpose(-2, -3)

        if chunk_size is not None:
            outer = self._chunk(a, b, chunk_size)
        else:
            outer = self._opm(a, b)

        # [*, N_res, N_res, 1]
        norm = torch.einsum("...abc,...adc->...bdc", mask, mask)
        norm = norm + self.eps

        # [*, N_res, N_res, C_z]
        if(inplace_safe):
            outer /= norm
        else:
            outer = outer / norm

        return outer

    def forward(self,
                m: torch.Tensor,
                mask: Optional[torch.Tensor] = None,
                chunk_size: Optional[int] = None,
                inplace_safe: bool = False,
    ) -> torch.Tensor:
        if(is_fp16_enabled()):
            with torch.cuda.amp.autocast(enabled=False):
                return self._forward(m.float(), mask, chunk_size, inplace_safe)
        else:
            return self._forward(m, mask, chunk_size, inplace_safe)

## algorithm 10:msa pair weighted averaging with gating

In [2]:
# Input projections

# pseudo msa and residue
N_seq = 50
N_res = 256

# channel information
cm = 64
cz = 128
c = 32
head = 8

# pseudo data
msa_input = torch.randn(N_seq,N_res,cm)
pair_representation = torch.rand(N_res,N_res,cz)

# model
layernorm1 = nn.LayerNorm(cm)
layernorm2 = nn.LayerNorm(cz)
linear1 = nn.Linear(cm,c*head,bias=False)
linear2 = nn.Linear(cz,head,bias=False)
linear3 = nn.Linear(cm,c*head,bias=False)
linear4 = nn.Linear(c*head,cm,bias=False)

# Input projection
msa = layernorm1(msa_input)
V = linear1(msa)
V = V.reshape(V.shape[:-1] + (head,-1)) # multiple head projection
bias = linear2(layernorm2(pair_representation))
gating = nn.Sigmoid()(linear3(msa))
gating = gating.reshape(gating.shape[:-1] + (head,-1)) # multiple head projection

# Weighted average with gating
weight = nn.Softmax(dim=-2)(bias) # N_res, N_res, head
output = torch.einsum("sih...,ijh,sjh...->sih...",gating,weight,V)

# Output projection
output = linear4(output.reshape(output.shape[:-2]+(-1,)))

In [4]:
# output : N_res,N_seq,c 
output_einsum = torch.einsum("sih...,ijh,sjh...->sih...",gating,weight,V)
output = torch.zeros(N_seq,N_res,head,c)


for s in range(output.shape[0]):
    for i in range(output.shape[1]):
        for h in range(output.shape[2]):
            output[s,i,h] = gating[s,i,h] * torch.sum(V[s,:,h,:]*weight[i,:,h].unsqueeze(1),dim=0)

In [9]:
print(torch.all((output_einsum - output) < 1e-5))

tensor(True)


In [111]:
(output_einsum[0,0] - output[0,0]) < 1e-5

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True])

## algorithm 12/13 Triangle updates of the pair representation

In [23]:
z = torch.randn(N_res,N_res,cz)
z = nn.LayerNorm(cz)(z)
a = nn.Sigmoid()(nn.Linear(cz,cz,bias=False)(z)) * nn.Linear(cz,cz,bias=False)(z)
b = nn.Sigmoid()(nn.Linear(cz,cz,bias=False)(z)) * nn.Linear(cz,cz,bias=False)(z)
g = nn.Sigmoid()(nn.Linear(cz,cz,bias=False)(z))


direction = "outging"

if direction == "outging":
    intermediate = nn.Linear(cz,cz,bias=False)(nn.LayerNorm(cz)(torch.einsum("ikc,jkc->ijc",a,b)))
elif direction == "incmoing":
    intermediate = nn.Linear(cz,cz,bias=False)(nn.LayerNorm(cz)(torch.einsum("kic,kjc->ijc",a,b)))
z_output = g * intermediate

In [27]:
direction = "incoming"


intermediate_incoming = nn.Linear(cz,cz,bias=False)(nn.LayerNorm(cz)(torch.einsum("kic,kjc->ijc",a,b)))
z_output_incoming = g * intermediate_incoming

In [46]:
test_outging = torch.einsum("ik...,jk...->ij...",a,b)
print(torch.all(test_outging[0,0] - torch.sum(a[0,] * b[0,],dim=0) < 1e-3))

tensor(True)


In [39]:
test_incoming = torch.einsum("ki...,kj...->ij...",a,b)
print(torch.all(test_incoming[0,0] - torch.sum(a[:,0] * b[:,0],dim=0) < 1e-3))

tensor(True)


## AlphaFold Multihead Attenntion

In [2]:
x = torch.randn(3,3)
y = torch.randn(3,3)
torch.einsum("ij,jk->ik",x,y)

tensor([[ 0.0558, -0.2889,  0.3755],
        [ 0.2219,  0.1194,  0.5650],
        [-0.2853, -1.7966, -0.1812]])

In [3]:
torch.einsum("ij,ik->ik",x,y)

tensor([[-0.6112,  1.1692, -1.8062],
        [-0.0200, -0.2070,  0.0071],
        [ 1.0505,  3.8545,  1.8491]])

In [15]:
(x.reshape(3,1,3)*y.reshape(3,3,1)).sum(dim=-1)

tensor([[-0.6112,  1.1692, -1.8062],
        [-0.0200, -0.2070,  0.0071],
        [ 1.0505,  3.8545,  1.8491]])

In [6]:
output = torch.zeros((3,3))
for i in range(output.shape[0]):
    for k in range(output.shape[1]):
        output[i,k] = torch.sum(x[i,] * y[i,k])
output

tensor([[-0.6112,  1.1692, -1.8062],
        [-0.0200, -0.2070,  0.0071],
        [ 1.0505,  3.8545,  1.8491]])

In [5]:
batch = 10 
N_res = 256
channel = 128
head = 8

pseudo_input = torch.randn(batch,N_res,channel)

linear1 = nn.Linear(channel,3*channel)
linear2 = nn.Linear(channel,channel)
softmax = nn.Softmax(dim=-1)

q,k,v = linear1(pseudo_input).chunk(3,dim=-1)
q = q.reshape(batch,N_res,head,int(channel/head)) * (channel/head) ** -0.5
k = k.reshape(batch,N_res,head,int(channel/head))
v = v.reshape(batch,N_res,head,int(channel/head))
attention1 = softmax(q.permute(0,2,1,3) @ k.permute(0,2,3,1))
attention2 = softmax(torch.einsum("bqhc,bkhc->bhqk",q,k))

output1 = (attention1 @ v.permute(0,2,1,3)).transpose(-2,-3)
output2 = torch.einsum("bhqk,bkhc->bqhc",attention2,v) # 之前的错误是写为了bhqk, bqhc；真正应该是bkhc

output1 = linear2(output1.reshape(output1.shape[:-2]+(-1,)))
output2 = linear2(output2.reshape(output2.shape[:-2]+(-1,)))

print(torch.allclose(attention1,attention2))
print(torch.allclose(output1,output2))

torch.Size([10, 256, 8, 16])
torch.Size([10, 256, 8, 16])
True
True


In [57]:
x = torch.arange(24).reshape(2,3,4)
print(x)
print(x.reshape(4,3,2))
print(x.permute(2,1,0))

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])


In [58]:
x.reshape(4,3,2)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]],

        [[18, 19],
         [20, 21],
         [22, 23]]])

In [59]:
x.permute(2,1,0)

tensor([[[ 0, 12],
         [ 4, 16],
         [ 8, 20]],

        [[ 1, 13],
         [ 5, 17],
         [ 9, 21]],

        [[ 2, 14],
         [ 6, 18],
         [10, 22]],

        [[ 3, 15],
         [ 7, 19],
         [11, 23]]])

In [7]:
activation_einsum = torch.einsum("bqhc,bkhc->bhqk", q, k)
activation_matrix = q.permute(0, 2, 1, 3) @ k.permute(0, 2, 1, 3).transpose(-1, -2)

# 确保两者相同
torch.allclose(activation_einsum, activation_matrix)

True

In [3]:
# activation
activation = torch.einsum("bqhc,bkhc->bhqk",q,k)
# output and concat 
output = torch.einsum("bhqk,bqhc->bqhc",activation,v)
output = output.reshape(batch,N_res,channel)



In [87]:
# matrix multiplication
activation_weight = q.reshape(batch,head,N_res,-1) @ k.reshape(batch,head,N_res,-1).transpose(-1,-2)

In [88]:
activation[0]

tensor([[[-0.6665,  0.6064, -0.4296,  ...,  0.3532, -0.1957,  0.5937],
         [ 0.2747, -0.4977, -0.1304,  ...,  0.0076, -0.3048, -0.9296],
         [ 0.0308, -0.1339,  0.3036,  ..., -0.4195,  0.3613, -0.0514],
         ...,
         [ 0.5664,  0.2680, -0.1534,  ...,  0.2680, -0.0338,  0.2619],
         [-0.5188, -0.1951,  1.1662,  ..., -0.3544, -0.3555,  0.2509],
         [-0.3540, -0.2852,  0.1981,  ...,  0.3805,  0.0887,  0.2447]],

        [[-0.3834,  0.1719, -0.1108,  ..., -0.4971, -0.1468, -0.3366],
         [ 0.1695,  0.7629,  0.0778,  ..., -0.3644,  0.4890, -0.3516],
         [ 0.1993,  0.2595,  0.7844,  ..., -0.6483, -0.2891,  0.4231],
         ...,
         [ 0.2987,  0.8410, -0.1231,  ...,  0.0992, -0.2299,  0.3941],
         [ 0.0879,  0.3421,  0.3264,  ..., -0.8061,  0.0335,  0.5958],
         [ 0.4242, -0.2689,  0.4354,  ..., -0.4684,  1.0080,  0.7794]]],
       grad_fn=<SelectBackward0>)

In [89]:
activation_weight[0]

tensor([[[-0.6665, -0.6025,  0.6064,  ...,  0.5723, -0.0917,  0.3571],
         [ 0.0574, -0.3834,  0.1375,  ..., -0.1645, -0.2602, -0.4735],
         [ 0.2747,  0.5407, -0.4977,  ..., -0.2870, -0.1773,  0.4674],
         ...,
         [-0.6115, -0.7624,  0.1468,  ...,  0.0752, -0.1521, -0.5720],
         [-0.0367, -0.2636, -0.3998,  ...,  0.7545, -0.3549, -0.4083],
         [-0.0781, -0.2383,  0.0311,  ..., -0.6561, -0.5898,  0.2084]],

        [[ 0.1004, -0.4839,  0.1916,  ...,  0.4732, -1.1515, -0.3070],
         [ 0.6639, -0.1203, -0.0753,  ..., -0.1722,  0.1032,  0.2047],
         [ 0.0483,  0.6451, -0.0687,  ..., -0.2258, -0.3715,  0.0644],
         ...,
         [-0.6793, -0.2143,  0.1027,  ...,  0.0335,  0.0279,  0.5958],
         [-0.3293,  0.1868, -0.2412,  ..., -0.5827,  0.2447, -0.1429],
         [-0.6114,  0.3001, -0.2557,  ...,  1.0080, -0.1627,  0.7794]]],
       grad_fn=<SelectBackward0>)

In [ ]:
class Attention(hk.Module):
  """Multihead attention."""

  def __init__(self, config, global_config, output_dim, name='attention'):
    super().__init__(name=name)

    self.config = config
    self.global_config = global_config
    self.output_dim = output_dim

  def __call__(self, q_data, m_data, mask, nonbatched_bias=None):
    """Builds Attention module.

    Arguments:
      q_data: A tensor of queries, shape [batch_size, N_queries, q_channels].
      m_data: A tensor of memories from which the keys and values are
        projected, shape [batch_size, N_keys, m_channels].
      mask: A mask for the attention, shape [batch_size, N_queries, N_keys].
      nonbatched_bias: Shared bias, shape [N_queries, N_keys].

    Returns:
      A float32 tensor of shape [batch_size, N_queries, output_dim].
    """
    # Sensible default for when the config keys are missing
    key_dim = self.config.get('key_dim', int(q_data.shape[-1]))
    value_dim = self.config.get('value_dim', int(m_data.shape[-1]))
    num_head = self.config.num_head
    assert key_dim % num_head == 0
    assert value_dim % num_head == 0
    key_dim = key_dim // num_head
    value_dim = value_dim // num_head

    q_weights = hk.get_parameter(
        'query_w', shape=(q_data.shape[-1], num_head, key_dim),
        dtype=q_data.dtype,
        init=glorot_uniform())
    k_weights = hk.get_parameter(
        'key_w', shape=(m_data.shape[-1], num_head, key_dim),
        dtype=q_data.dtype,
        init=glorot_uniform())
    v_weights = hk.get_parameter(
        'value_w', shape=(m_data.shape[-1], num_head, value_dim),
        dtype=q_data.dtype,
        init=glorot_uniform())

    q = jnp.einsum('bqa,ahc->bqhc', q_data, q_weights) * key_dim**(-0.5)
    k = jnp.einsum('bka,ahc->bkhc', m_data, k_weights)
    v = jnp.einsum('bka,ahc->bkhc', m_data, v_weights)
    logits = jnp.einsum('bqhc,bkhc->bhqk', q, k)
    if nonbatched_bias is not None:
      logits += jnp.expand_dims(nonbatched_bias, axis=0)
    logits = jnp.where(mask, logits, _SOFTMAX_MASK)
    weights = utils.stable_softmax(logits)
    weighted_avg = jnp.einsum('bhqk,bkhc->bqhc', weights, v)

    if self.global_config.zero_init:
      init = hk.initializers.Constant(0.0)
    else:
      init = glorot_uniform()

    if self.config.gating:
      gating_weights = hk.get_parameter(
          'gating_w',
          shape=(q_data.shape[-1], num_head, value_dim),
          dtype=q_data.dtype,
          init=hk.initializers.Constant(0.0))
      gating_bias = hk.get_parameter(
          'gating_b',
          shape=(num_head, value_dim),
          dtype=q_data.dtype,
          init=hk.initializers.Constant(1.0))

      gate_values = jnp.einsum('bqc, chv->bqhv', q_data,
                               gating_weights) + gating_bias

      gate_values = jax.nn.sigmoid(gate_values)

      weighted_avg *= gate_values

    o_weights = hk.get_parameter(
        'output_w', shape=(num_head, value_dim, self.output_dim),
        dtype=q_data.dtype,
        init=init)
    o_bias = hk.get_parameter(
        'output_b', shape=(self.output_dim,),
        dtype=q_data.dtype,
        init=hk.initializers.Constant(0.0))

    output = jnp.einsum('bqhc,hco->bqo', weighted_avg, o_weights) + o_bias

    return output

In [ ]:
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig

# This will prompt you to get an API key from huggingface hub, make one with
# "Read" or "Write" permission and copy it back here.
login()

# This will download the model weights and instantiate the model on your machine.
model: ESM3InferenceClient = ESM3.from_pretrained(model_name="esm3_sm_open_v1").to("cuda") # or "cpu"

# Generate a completion for a partial Carbonic Anhydrase (2vvb)
prompt = "___________________________________________________DQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFHFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPP___________________________________________________________"
protein = ESMProtein(sequence=prompt)
# Generate the sequence, then the structure. This will iteratively unmask the sequence track.
protein = model.generate(protein, GenerationConfig(track="sequence", num_steps=8, temperature=0.7))
# We can show the predicted structure for the generated sequence.
protein = model.generate(protein, GenerationConfig(track="structure", num_steps=8))
protein.to_pdb("./generation.pdb")
# Then we can do a round trip design by inverse folding the sequence and recomputing the structure
protein.sequence = None
protein = model.generate(protein, GenerationConfig(track="sequence", num_steps=8))
protein.structure = None
protein = model.generate(protein, GenerationConfig(track="structure", num_steps=8))
protein.to_pdb("./round_tripped.pdb")

In [15]:
import collections
import itertools
from typing import Set

def _keep_line(line: str, seqnames: Set[str]) -> bool:
  """Function to decide which lines to keep."""
  if not line.strip():
    return True
  if line.strip() == '//':  # End tag
    return True
  if line.startswith('# STOCKHOLM'):  # Start tag
    return True
  if line.startswith('#=GC RF'):  # Reference Annotation Line
    return True
  if line[:4] == '#=GS':  # Description lines - keep if sequence in list.
    _, seqname, _ = line.split(maxsplit=2)
    return seqname in seqnames
  elif line.startswith('#'):  # Other markup - filter out
    return False
  else:  # Alignment data - keep if sequence in list.
    seqname = line.partition(' ')[0]
    return seqname in seqnames

def deduplicate_stockholm_msa(stockholm_msa: str) -> str:
  """Remove duplicate sequences (ignoring insertions wrt query)."""
  sequence_dict = collections.defaultdict(str)

  # First we must extract all sequences from the MSA.
  for line in stockholm_msa.splitlines():
    # Only consider the alignments - ignore reference annotation, empty lines,
    # descriptions or markup.
    if line.strip() and not line.startswith(('#', '//')):
      line = line.strip()
      seqname, alignment = line.split()
      sequence_dict[seqname] += alignment

  seen_sequences = set()
  seqnames = set()
  # First alignment is the query.
  query_align = next(iter(sequence_dict.values()))
  mask = [c != '-' for c in query_align]  # Mask is False for insertions.
  for seqname, alignment in sequence_dict.items():
    # Apply mask to remove all insertions from the string.
    masked_alignment = ''.join(itertools.compress(alignment, mask))
    if masked_alignment in seen_sequences:
      continue
    else:
      seen_sequences.add(masked_alignment)
      seqnames.add(seqname)

  filtered_lines = []
  for line in stockholm_msa.splitlines():
    if _keep_line(line, seqnames):
      filtered_lines.append(line)

  return '\n'.join(filtered_lines) + '\n'

def remove_empty_columns_from_stockholm_msa(stockholm_msa: str) -> str:
  """Removes empty columns (dashes-only) from a Stockholm MSA."""
  processed_lines = {}
  unprocessed_lines = {}
  for i, line in enumerate(stockholm_msa.splitlines()):
    if line.startswith('#=GC RF'):
      reference_annotation_i = i
      reference_annotation_line = line
      # Reached the end of this chunk of the alignment. Process chunk.
      _, _, first_alignment = line.rpartition(' ')
      mask = []
      for j in range(len(first_alignment)):
        for _, unprocessed_line in unprocessed_lines.items():
          prefix, _, alignment = unprocessed_line.rpartition(' ')
          if alignment[j] != '-':
            mask.append(True)
            break
        else:  # Every row contained a hyphen - empty column.
          mask.append(False)
      # Add reference annotation for processing with mask.
      unprocessed_lines[reference_annotation_i] = reference_annotation_line

      if not any(mask):  # All columns were empty. Output empty lines for chunk.
        for line_index in unprocessed_lines:
          processed_lines[line_index] = ''
      else:
        for line_index, unprocessed_line in unprocessed_lines.items():
          prefix, _, alignment = unprocessed_line.rpartition(' ')
          masked_alignment = ''.join(itertools.compress(alignment, mask))
          processed_lines[line_index] = f'{prefix} {masked_alignment}'

      # Clear raw_alignments.
      unprocessed_lines = {}
    elif line.strip() and not line.startswith(('#', '//')):
      unprocessed_lines[i] = line
    else:
      processed_lines[i] = line
  return '\n'.join((processed_lines[i] for i in range(len(processed_lines))))



In [19]:
with open("/home/sirius/PhD/software/alphafold/output/7s4h/msas/D/uniref90_hits.sto") as f:
    stockholm_msa = f.read()

In [20]:
x = deduplicate_stockholm_msa(stockholm_msa)

In [21]:
x = remove_empty_columns_from_stockholm_msa(x)

In [29]:
# Copyright 2021 DeepMind Technologies Limited
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Functions for parsing various file formats."""
import collections
import dataclasses
import itertools
import re
import string
from typing import Dict, Iterable, List, Optional, Sequence, Tuple, Set

# Internal import (7716).


DeletionMatrix = Sequence[Sequence[int]]


@dataclasses.dataclass(frozen=True)
class Msa:
  """Class representing a parsed MSA file."""
  sequences: Sequence[str]
  deletion_matrix: DeletionMatrix
  descriptions: Sequence[str]

  def __post_init__(self):
    if not (len(self.sequences) ==
            len(self.deletion_matrix) ==
            len(self.descriptions)):
      raise ValueError(
          'All fields for an MSA must have the same length. '
          f'Got {len(self.sequences)} sequences, '
          f'{len(self.deletion_matrix)} rows in the deletion matrix and '
          f'{len(self.descriptions)} descriptions.')

  def __len__(self):
    return len(self.sequences)

  def truncate(self, max_seqs: int):
    return Msa(sequences=self.sequences[:max_seqs],
               deletion_matrix=self.deletion_matrix[:max_seqs],
               descriptions=self.descriptions[:max_seqs])


@dataclasses.dataclass(frozen=True)
class TemplateHit:
  """Class representing a template hit."""
  index: int
  name: str
  aligned_cols: int
  sum_probs: Optional[float]
  query: str
  hit_sequence: str
  indices_query: List[int]
  indices_hit: List[int]


def parse_fasta(fasta_string: str) -> Tuple[Sequence[str], Sequence[str]]:
  """Parses FASTA string and returns list of strings with amino-acid sequences.

  Arguments:
    fasta_string: The string contents of a FASTA file.

  Returns:
    A tuple of two lists:
    * A list of sequences.
    * A list of sequence descriptions taken from the comment lines. In the
      same order as the sequences.
  """
  sequences = []
  descriptions = []
  index = -1
  for line in fasta_string.splitlines():
    line = line.strip()
    if line.startswith('>'):
      index += 1
      descriptions.append(line[1:])  # Remove the '>' at the beginning.
      sequences.append('')
      continue
    elif not line:
      continue  # Skip blank lines.
    sequences[index] += line

  return sequences, descriptions


def parse_stockholm(stockholm_string: str) -> Msa:
  """Parses sequences and deletion matrix from stockholm format alignment.

  Args:
    stockholm_string: The string contents of a stockholm file. The first
      sequence in the file should be the query sequence.

  Returns:
    A tuple of:
      * A list of sequences that have been aligned to the query. These
        might contain duplicates.
      * The deletion matrix for the alignment as a list of lists. The element
        at `deletion_matrix[i][j]` is the number of residues deleted from
        the aligned sequence i at residue position j.
      * The names of the targets matched, including the jackhmmer subsequence
        suffix.
  """
  name_to_sequence = collections.OrderedDict()
  for line in stockholm_string.splitlines():
    line = line.strip()
    if not line or line.startswith(('#', '//')):
      continue
    name, sequence = line.split()
    if name not in name_to_sequence:
      name_to_sequence[name] = ''
    name_to_sequence[name] += sequence

  msa = []
  deletion_matrix = []

  query = ''
  keep_columns = []
  for seq_index, sequence in enumerate(name_to_sequence.values()):
    if seq_index == 0:
      # Gather the columns with gaps from the query
      query = sequence
      keep_columns = [i for i, res in enumerate(query) if res != '-']

    # Remove the columns with gaps in the query from all sequences.
    aligned_sequence = ''.join([sequence[c] for c in keep_columns])

    msa.append(aligned_sequence)

    # Count the number of deletions w.r.t. query.
    deletion_vec = []
    deletion_count = 0
    for seq_res, query_res in zip(sequence, query):
      if seq_res != '-' or query_res != '-':
        if query_res == '-':
          deletion_count += 1
        else:
          deletion_vec.append(deletion_count)
          deletion_count = 0
    deletion_matrix.append(deletion_vec)

  return Msa(sequences=msa,
             deletion_matrix=deletion_matrix,
             descriptions=list(name_to_sequence.keys()))


def parse_a3m(a3m_string: str) -> Msa:
  """Parses sequences and deletion matrix from a3m format alignment.

  Args:
    a3m_string: The string contents of a a3m file. The first sequence in the
      file should be the query sequence.

  Returns:
    A tuple of:
      * A list of sequences that have been aligned to the query. These
        might contain duplicates.
      * The deletion matrix for the alignment as a list of lists. The element
        at `deletion_matrix[i][j]` is the number of residues deleted from
        the aligned sequence i at residue position j.
      * A list of descriptions, one per sequence, from the a3m file.
  """
  sequences, descriptions = parse_fasta(a3m_string)
  deletion_matrix = []
  for msa_sequence in sequences:
    deletion_vec = []
    deletion_count = 0
    for j in msa_sequence:
      if j.islower():
        deletion_count += 1
      else:
        deletion_vec.append(deletion_count)
        deletion_count = 0
    deletion_matrix.append(deletion_vec)

  # Make the MSA matrix out of aligned (deletion-free) sequences.
  deletion_table = str.maketrans('', '', string.ascii_lowercase)
  aligned_sequences = [s.translate(deletion_table) for s in sequences]
  return Msa(sequences=aligned_sequences,
             deletion_matrix=deletion_matrix,
             descriptions=descriptions)


def _convert_sto_seq_to_a3m(
    query_non_gaps: Sequence[bool], sto_seq: str) -> Iterable[str]:
  for is_query_res_non_gap, sequence_res in zip(query_non_gaps, sto_seq):
    if is_query_res_non_gap:
      yield sequence_res
    elif sequence_res != '-':
      yield sequence_res.lower()


def convert_stockholm_to_a3m(stockholm_format: str,
                             max_sequences: Optional[int] = None,
                             remove_first_row_gaps: bool = True) -> str:
  """Converts MSA in Stockholm format to the A3M format."""
  descriptions = {}
  sequences = {}
  reached_max_sequences = False

  for line in stockholm_format.splitlines():
    reached_max_sequences = max_sequences and len(sequences) >= max_sequences
    if line.strip() and not line.startswith(('#', '//')):
      # Ignore blank lines, markup and end symbols - remainder are alignment
      # sequence parts.
      seqname, aligned_seq = line.split(maxsplit=1)
      if seqname not in sequences:
        if reached_max_sequences:
          continue
        sequences[seqname] = ''
      sequences[seqname] += aligned_seq

  for line in stockholm_format.splitlines():
    if line[:4] == '#=GS':
      # Description row - example format is:
      # #=GS UniRef90_Q9H5Z4/4-78            DE [subseq from] cDNA: FLJ22755 ...
      columns = line.split(maxsplit=3)
      seqname, feature = columns[1:3]
      value = columns[3] if len(columns) == 4 else ''
      if feature != 'DE':
        continue
      if reached_max_sequences and seqname not in sequences:
        continue
      descriptions[seqname] = value
      if len(descriptions) == len(sequences):
        break

  # Convert sto format to a3m line by line
  a3m_sequences = {}
  if remove_first_row_gaps:
    # query_sequence is assumed to be the first sequence
    query_sequence = next(iter(sequences.values()))
    query_non_gaps = [res != '-' for res in query_sequence]
  for seqname, sto_sequence in sequences.items():
    # Dots are optional in a3m format and are commonly removed.
    out_sequence = sto_sequence.replace('.', '')
    if remove_first_row_gaps:
      out_sequence = ''.join(
          _convert_sto_seq_to_a3m(query_non_gaps, out_sequence))
    a3m_sequences[seqname] = out_sequence

  fasta_chunks = (f">{k} {descriptions.get(k, '')}\n{a3m_sequences[k]}"
                  for k in a3m_sequences)
  return '\n'.join(fasta_chunks) + '\n'  # Include terminating newline.


def _keep_line(line: str, seqnames: Set[str]) -> bool:
  """Function to decide which lines to keep."""
  if not line.strip():
    return True
  if line.strip() == '//':  # End tag
    return True
  if line.startswith('# STOCKHOLM'):  # Start tag
    return True
  if line.startswith('#=GC RF'):  # Reference Annotation Line
    return True
  if line[:4] == '#=GS':  # Description lines - keep if sequence in list.
    _, seqname, _ = line.split(maxsplit=2)
    return seqname in seqnames
  elif line.startswith('#'):  # Other markup - filter out
    return False
  else:  # Alignment data - keep if sequence in list.
    seqname = line.partition(' ')[0]
    return seqname in seqnames


def truncate_stockholm_msa(stockholm_msa_path: str, max_sequences: int) -> str:
  """Reads + truncates a Stockholm file while preventing excessive RAM usage."""
  seqnames = set()
  filtered_lines = []

  with open(stockholm_msa_path) as f:
    for line in f:
      if line.strip() and not line.startswith(('#', '//')):
        # Ignore blank lines, markup and end symbols - remainder are alignment
        # sequence parts.
        seqname = line.partition(' ')[0]
        seqnames.add(seqname)
        if len(seqnames) >= max_sequences:
          break

    f.seek(0)
    for line in f:
      if _keep_line(line, seqnames):
        filtered_lines.append(line)

  return ''.join(filtered_lines)


def remove_empty_columns_from_stockholm_msa(stockholm_msa: str) -> str:
  """Removes empty columns (dashes-only) from a Stockholm MSA."""
  processed_lines = {}
  unprocessed_lines = {}
  for i, line in enumerate(stockholm_msa.splitlines()):
    if line.startswith('#=GC RF'):
      reference_annotation_i = i
      reference_annotation_line = line
      # Reached the end of this chunk of the alignment. Process chunk.
      _, _, first_alignment = line.rpartition(' ')
      mask = []
      for j in range(len(first_alignment)):
        for _, unprocessed_line in unprocessed_lines.items():
          prefix, _, alignment = unprocessed_line.rpartition(' ')
          if alignment[j] != '-':
            mask.append(True)
            break
        else:  # Every row contained a hyphen - empty column.
          mask.append(False)
      # Add reference annotation for processing with mask.
      unprocessed_lines[reference_annotation_i] = reference_annotation_line

      if not any(mask):  # All columns were empty. Output empty lines for chunk.
        for line_index in unprocessed_lines:
          processed_lines[line_index] = ''
      else:
        for line_index, unprocessed_line in unprocessed_lines.items():
          prefix, _, alignment = unprocessed_line.rpartition(' ')
          masked_alignment = ''.join(itertools.compress(alignment, mask))
          processed_lines[line_index] = f'{prefix} {masked_alignment}'

      # Clear raw_alignments.
      unprocessed_lines = {}
    elif line.strip() and not line.startswith(('#', '//')):
      unprocessed_lines[i] = line
    else:
      processed_lines[i] = line
  return '\n'.join((processed_lines[i] for i in range(len(processed_lines))))


def deduplicate_stockholm_msa(stockholm_msa: str) -> str:
  """Remove duplicate sequences (ignoring insertions wrt query)."""
  sequence_dict = collections.defaultdict(str)

  # First we must extract all sequences from the MSA.
  for line in stockholm_msa.splitlines():
    # Only consider the alignments - ignore reference annotation, empty lines,
    # descriptions or markup.
    if line.strip() and not line.startswith(('#', '//')):
      line = line.strip()
      seqname, alignment = line.split()
      sequence_dict[seqname] += alignment

  seen_sequences = set()
  seqnames = set()
  # First alignment is the query.
  query_align = next(iter(sequence_dict.values()))
  mask = [c != '-' for c in query_align]  # Mask is False for insertions.
  for seqname, alignment in sequence_dict.items():
    # Apply mask to remove all insertions from the string.
    masked_alignment = ''.join(itertools.compress(alignment, mask))
    if masked_alignment in seen_sequences:
      continue
    else:
      seen_sequences.add(masked_alignment)
      seqnames.add(seqname)

  filtered_lines = []
  for line in stockholm_msa.splitlines():
    if _keep_line(line, seqnames):
      filtered_lines.append(line)

  return '\n'.join(filtered_lines) + '\n'


def _get_hhr_line_regex_groups(
    regex_pattern: str, line: str) -> Sequence[Optional[str]]:
  match = re.match(regex_pattern, line)
  if match is None:
    raise RuntimeError(f'Could not parse query line {line}')
  return match.groups()


def _update_hhr_residue_indices_list(
    sequence: str, start_index: int, indices_list: List[int]):
  """Computes the relative indices for each residue with respect to the original sequence."""
  counter = start_index
  for symbol in sequence:
    if symbol == '-':
      indices_list.append(-1)
    else:
      indices_list.append(counter)
      counter += 1


def _parse_hhr_hit(detailed_lines: Sequence[str]) -> TemplateHit:
  """Parses the detailed HMM HMM comparison section for a single Hit.

  This works on .hhr files generated from both HHBlits and HHSearch.

  Args:
    detailed_lines: A list of lines from a single comparison section between 2
      sequences (which each have their own HMM's)

  Returns:
    A dictionary with the information from that detailed comparison section

  Raises:
    RuntimeError: If a certain line cannot be processed
  """
  # Parse first 2 lines.
  number_of_hit = int(detailed_lines[0].split()[-1])
  name_hit = detailed_lines[1][1:]

  # Parse the summary line.
  pattern = (
      'Probab=(.*)[\t ]*E-value=(.*)[\t ]*Score=(.*)[\t ]*Aligned_cols=(.*)[\t'
      ' ]*Identities=(.*)%[\t ]*Similarity=(.*)[\t ]*Sum_probs=(.*)[\t '
      ']*Template_Neff=(.*)')
  match = re.match(pattern, detailed_lines[2])
  if match is None:
    raise RuntimeError(
        'Could not parse section: %s. Expected this: \n%s to contain summary.' %
        (detailed_lines, detailed_lines[2]))
  (_, _, _, aligned_cols, _, _, sum_probs, _) = [float(x)
                                                 for x in match.groups()]

  # The next section reads the detailed comparisons. These are in a 'human
  # readable' format which has a fixed length. The strategy employed is to
  # assume that each block starts with the query sequence line, and to parse
  # that with a regexp in order to deduce the fixed length used for that block.
  query = ''
  hit_sequence = ''
  indices_query = []
  indices_hit = []
  length_block = None

  for line in detailed_lines[3:]:
    # Parse the query sequence line
    if (line.startswith('Q ') and not line.startswith('Q ss_dssp') and
        not line.startswith('Q ss_pred') and
        not line.startswith('Q Consensus')):
      # Thus the first 17 characters must be 'Q <query_name> ', and we can parse
      # everything after that.
      #              start    sequence       end       total_sequence_length
      patt = r'[\t ]*([0-9]*) ([A-Z-]*)[\t ]*([0-9]*) \([0-9]*\)'
      groups = _get_hhr_line_regex_groups(patt, line[17:])

      # Get the length of the parsed block using the start and finish indices,
      # and ensure it is the same as the actual block length.
      start = int(groups[0]) - 1  # Make index zero based.
      delta_query = groups[1]
      end = int(groups[2])
      num_insertions = len([x for x in delta_query if x == '-'])
      length_block = end - start + num_insertions
      assert length_block == len(delta_query)

      # Update the query sequence and indices list.
      query += delta_query
      _update_hhr_residue_indices_list(delta_query, start, indices_query)

    elif line.startswith('T '):
      # Parse the hit sequence.
      if (not line.startswith('T ss_dssp') and
          not line.startswith('T ss_pred') and
          not line.startswith('T Consensus')):
        # Thus the first 17 characters must be 'T <hit_name> ', and we can
        # parse everything after that.
        #              start    sequence       end     total_sequence_length
        patt = r'[\t ]*([0-9]*) ([A-Z-]*)[\t ]*[0-9]* \([0-9]*\)'
        groups = _get_hhr_line_regex_groups(patt, line[17:])
        start = int(groups[0]) - 1  # Make index zero based.
        delta_hit_sequence = groups[1]
        assert length_block == len(delta_hit_sequence)

        # Update the hit sequence and indices list.
        hit_sequence += delta_hit_sequence
        _update_hhr_residue_indices_list(delta_hit_sequence, start, indices_hit)

  return TemplateHit(
      index=number_of_hit,
      name=name_hit,
      aligned_cols=int(aligned_cols),
      sum_probs=sum_probs,
      query=query,
      hit_sequence=hit_sequence,
      indices_query=indices_query,
      indices_hit=indices_hit,
  )


def parse_hhr(hhr_string: str) -> Sequence[TemplateHit]:
  """Parses the content of an entire HHR file."""
  lines = hhr_string.splitlines()

  # Each .hhr file starts with a results table, then has a sequence of hit
  # "paragraphs", each paragraph starting with a line 'No <hit number>'. We
  # iterate through each paragraph to parse each hit.

  block_starts = [i for i, line in enumerate(lines) if line.startswith('No ')]

  hits = []
  if block_starts:
    block_starts.append(len(lines))  # Add the end of the final block.
    for i in range(len(block_starts) - 1):
      hits.append(_parse_hhr_hit(lines[block_starts[i]:block_starts[i + 1]]))
  return hits


def parse_e_values_from_tblout(tblout: str) -> Dict[str, float]:
  """Parse target to e-value mapping parsed from Jackhmmer tblout string."""
  e_values = {'query': 0}
  lines = [line for line in tblout.splitlines() if line[0] != '#']
  # As per http://eddylab.org/software/hmmer/Userguide.pdf fields are
  # space-delimited. Relevant fields are (1) target name:  and
  # (5) E-value (full sequence) (numbering from 1).
  for line in lines:
    fields = line.split()
    e_value = fields[4]
    target_name = fields[0]
    e_values[target_name] = float(e_value)
  return e_values


def _get_indices(sequence: str, start: int) -> List[int]:
  """Returns indices for non-gap/insert residues starting at the given index."""
  indices = []
  counter = start
  for symbol in sequence:
    # Skip gaps but add a placeholder so that the alignment is preserved.
    if symbol == '-':
      indices.append(-1)
    # Skip deleted residues, but increase the counter.
    elif symbol.islower():
      counter += 1
    # Normal aligned residue. Increase the counter and append to indices.
    else:
      indices.append(counter)
      counter += 1
  return indices


@dataclasses.dataclass(frozen=True)
class HitMetadata:
  pdb_id: str
  chain: str
  start: int
  end: int
  length: int
  text: str


def _parse_hmmsearch_description(description: str) -> HitMetadata:
  """Parses the hmmsearch A3M sequence description line."""
  # Example 1: >4pqx_A/2-217 [subseq from] mol:protein length:217  Free text
  # Example 2: >5g3r_A/1-55 [subseq from] mol:protein length:352
  match = re.match(
      r'^>?([a-z0-9]+)_(\w+)/([0-9]+)-([0-9]+).*protein length:([0-9]+) *(.*)$',
      description.strip())

  if not match:
    raise ValueError(f'Could not parse description: "{description}".')

  return HitMetadata(
      pdb_id=match[1],
      chain=match[2],
      start=int(match[3]),
      end=int(match[4]),
      length=int(match[5]),
      text=match[6])


def parse_hmmsearch_a3m(query_sequence: str,
                        a3m_string: str,
                        skip_first: bool = True) -> Sequence[TemplateHit]:
  """Parses an a3m string produced by hmmsearch.

  Args:
    query_sequence: The query sequence.
    a3m_string: The a3m string produced by hmmsearch.
    skip_first: Whether to skip the first sequence in the a3m string.

  Returns:
    A sequence of `TemplateHit` results.
  """
  # Zip the descriptions and MSAs together, skip the first query sequence.
  parsed_a3m = list(zip(*parse_fasta(a3m_string)))
  if skip_first:
    parsed_a3m = parsed_a3m[1:]

  indices_query = _get_indices(query_sequence, start=0)

  hits = []
  for i, (hit_sequence, hit_description) in enumerate(parsed_a3m, start=1):
    if 'mol:protein' not in hit_description:
      continue  # Skip non-protein chains.
    metadata = _parse_hmmsearch_description(hit_description)
    # Aligned columns are only the match states.
    aligned_cols = sum([r.isupper() and r != '-' for r in hit_sequence])
    indices_hit = _get_indices(hit_sequence, start=metadata.start - 1)

    hit = TemplateHit(
        index=i,
        name=f'{metadata.pdb_id}_{metadata.chain}',
        aligned_cols=aligned_cols,
        sum_probs=None,
        query=query_sequence,
        hit_sequence=hit_sequence.upper(),
        indices_query=indices_query,
        indices_hit=indices_hit,
    )
    hits.append(hit)

  return hits


In [33]:
with open("/home/sirius/PhD/software/alphafold/output/MpGDH/msas/pdb_hits.hhr","r") as f:
    hhr_string = f.read()
y = parse_hhr(hhr_string)

In [36]:
y[-1]

TemplateHit(index=500, name='2ZXI_D tRNA uridine 5-carboxymethylaminomethyl modification enzyme; modification, tRNA, 5-carboxymethylaminomethyl uridine, wobble; HET: FAD; 2.3A {Aquifex aeolicus}', aligned_cols=39, sum_probs=34.8, query='SSSTDTYDYVIVGGGVAGLALASRISENKDVTVAVLESGP', hit_sequence='AWVVDEFDVVVIGGGHAGIEAALAAAR-MGAKTAMFVLNA', indices_query=[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44], indices_hit=[21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, -1, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])

In [45]:
y[-1].aligned_cols

39

In [38]:
y[-1].query

'SSSTDTYDYVIVGGGVAGLALASRISENKDVTVAVLESGP'

In [39]:
y[-1].hit_sequence

'AWVVDEFDVVVIGGGHAGIEAALAAAR-MGAKTAMFVLNA'

In [41]:
len("AWVVDEFDVVVIGGGHAGIEAALAAAR-MGAKTAMFVLNA")

40

In [43]:
y[-1].indices_query.__len__()

40

In [44]:
y[-1].indices_hit.__len__()

40

In [55]:
y[-1].name

id_match = re.match(r'[a-zA-Z\d]{4}_[a-zA-Z0-9.]+', y[-1].name)
if not id_match:
    raise ValueError(f'hit.name did not start with PDBID_chain: {y[-1].name}')
pdb_id, chain_id = id_match.group(0).split('_')
print(pdb_id.lower(), chain_id)

2zxi D


In [46]:
import numpy as np

feature = np.load("/home/sirius/PhD/software/alphafold/output/MpGDH/features.pkl",allow_pickle=True)


In [51]:
feature['residue_index'].shape

(621,)

In [52]:
for key in feature:
    if key.startswith("template"):
        print(key,feature[key].shape)

template_aatype (20, 621, 22)
template_all_atom_masks (20, 621, 37)
template_all_atom_positions (20, 621, 37, 3)
template_domain_names (20,)
template_sequence (20,)
template_sum_probs (20, 1)


In [100]:
feature['template_sequence'][-1]

b'-------DNLTGDIVIIGAGAAGSLLAHYLARFSNMKIILLEAGHSHFNDPVVTDPMGFFSMQNPSYSWQGAQEPNTGAYGNRPIIAHGMGFGGSTMINRLNLVVGGRTVFDDWP----VGWKYDDVKNYFRRVLVDINPVRD--------NTKASITSVALDASGEDFLLNKATGNVPNVEKTTPDAVPDYEGVSVVAFSSFYNQLSDGNYIRKYAGNTYLNRNYKFSGLRVVSDAVVDRIIFK-G-----NRAVGVNYIDRE--GIMHYVKVNKEVVVTSGAFYTPTILQRSGIGDFTYLSSIGVKLVYNNPLVGTGLKNHYSPVTITRVHGEPSEVSRF------L----SNMAA-NPTNMGFKGLAELGFHR-----LD----PN-----KPA----NAN---T----------------------VTYRKYQLMMTAGVGILSGLSPSSNNLFTLIADDIRFAPEGYIKIGTPIPRDVPKIFFNTFVTPADQQWLISGYDIIYQTLMSMNARNDLIWHYFVPTLVGDTLSKLSYYPRVGAHLDSHQGCSCSIGR-----TVDSNLKVIGTQNVRVADLSAAAFPPGGNTWATASMIGARAVDLILGFPYLRDL---'

In [56]:
def _parse_obsolete(obsolete_file_path: str) :
  """Parses the data file from PDB that lists which pdb_ids are obsolete."""
  with open(obsolete_file_path) as f:
    result = {}
    for line in f:
      line = line.strip()
      # Format:    Date      From     To
      # 'OBSLTE    06-NOV-19 6G9Y'                - Removed, rare
      # 'OBSLTE    31-JUL-94 116L     216L'       - Replaced, common
      # 'OBSLTE    26-SEP-06 2H33     2JM5 2OWI'  - Replaced by multiple, rare
      if line.startswith('OBSLTE'):
        if len(line) > 30:
          # Replaced by at least one structure.
          from_id = line[20:24].lower()
          to_id = line[29:33].lower()
          result[from_id] = to_id
        elif len(line) == 24:
          # Removed.
          from_id = line[20:24].lower()
          result[from_id] = None
    return result

In [57]:
obsolete_pdbs_path = "/home/sirius/PhD/database/pdb_mmcif/obsolete.dat"
_obsolete_pdbs = _parse_obsolete(obsolete_pdbs_path)
# if this pdb was removed, don't use it

In [58]:
from typing import Mapping
def _parse_release_dates(path: str) -> Mapping[str, datetime.datetime]:
  """Parses release dates file, returns a mapping from PDBs to release dates."""
  if path.endswith('txt'):
    release_dates = {}
    with open(path, 'r') as f:
      for line in f:
        pdb_id, date = line.split(':')
        date = date.strip()
        # Python 3.6 doesn't have datetime.date.fromisoformat() which is about
        # 90x faster than strptime. However, splitting the string manually is
        # about 10x faster than strptime.
        release_dates[pdb_id.strip()] = datetime.datetime(
            year=int(date[:4]), month=int(date[5:7]), day=int(date[8:10]))
    return release_dates
  else:
    raise ValueError('Invalid format of the release date file %s.' % path)


_release_dates = _parse_release_dates(release_dates_path)

{'116l': '216l',
 '125d': '1aw6',
 '14ps': '1qjb',
 '151c': '251c',
 '156b': '256b',
 '179l': None,
 '1a0v': '1y46',
 '1a0w': '1y4f',
 '1a0x': '1y4g',
 '1a0y': '1y4p',
 '1a10': None,
 '1a1y': None,
 '1a2k': '5bxq',
 '1a6o': '1lr4',
 '1a9d': '1bz9',
 '1a9k': '1b0g',
 '1aa8': '1ve9',
 '1aak': '2aak',
 '1abh': '2abh',
 '1abk': '2abk',
 '1abm': '1n0j',
 '1abp': '1abe',
 '1abx': '2abx',
 '1ace': '2ace',
 '1ack': '2ack',
 '1act': '2act',
 '1ada': '2ada',
 '1adh': None,
 '1adk': '2adk',
 '1adm': '2adm',
 '1adt': '2wb0',
 '1afg': '2afg',
 '1afn': '2afn',
 '1ak3': '2ak3',
 '1alo': '1hlr',
 '1alp': '2alp',
 '1alr': '2alr',
 '1am3': '1a8o',
 '1amg': '2amg',
 '1amv': '2amv',
 '1anh': '2anh',
 '1ape': '2ape',
 '1apg': '3rtj',
 '1app': '2app',
 '1apr': '2apr',
 '1ar3': '1b0j',
 '1ara': '1am0',
 '1arn': '1dyz',
 '1as9': '1b0k',
 '1asi': '2asi',
 '1at7': '1bax',
 '1atc': '3atc',
 '1atq': '1b0m',
 '1aub': '1e0e',
 '1axf': '1y85',
 '1ayh': '2ayh',
 '1ayq': '2ayq',
 '1az9': '1wl9',
 '1aza': '2aza',
 '1b1

In [72]:
# Copyright 2021 DeepMind Technologies Limited
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Parses the mmCIF file format."""
import collections
import dataclasses
import functools
import io
from typing import Any, Mapping, Optional, Sequence, Tuple

from absl import logging
from Bio import PDB
import Bio

# Type aliases:
ChainId = str
PdbHeader = Mapping[str, Any]
PdbStructure = PDB.Structure.Structure
SeqRes = str
MmCIFDict = Mapping[str, Sequence[str]]


@dataclasses.dataclass(frozen=True)
class Monomer:
  id: str
  num: int


# Note - mmCIF format provides no guarantees on the type of author-assigned
# sequence numbers. They need not be integers.
@dataclasses.dataclass(frozen=True)
class AtomSite:
  residue_name: str
  author_chain_id: str
  mmcif_chain_id: str
  author_seq_num: str
  mmcif_seq_num: int
  insertion_code: str
  hetatm_atom: str
  model_num: int


# Used to map SEQRES index to a residue in the structure.
@dataclasses.dataclass(frozen=True)
class ResiduePosition:
  chain_id: str
  residue_number: int
  insertion_code: str


@dataclasses.dataclass(frozen=True)
class ResidueAtPosition:
  position: Optional[ResiduePosition]
  name: str
  is_missing: bool
  hetflag: str


@dataclasses.dataclass(frozen=True)
class MmcifObject:
  """Representation of a parsed mmCIF file.

  Contains:
    file_id: A meaningful name, e.g. a pdb_id. Should be unique amongst all
      files being processed.
    header: Biopython header.
    structure: Biopython structure.
    chain_to_seqres: Dict mapping chain_id to 1 letter amino acid sequence. E.g.
      {'A': 'ABCDEFG'}
    seqres_to_structure: Dict; for each chain_id contains a mapping between
      SEQRES index and a ResidueAtPosition. e.g. {'A': {0: ResidueAtPosition,
                                                        1: ResidueAtPosition,
                                                        ...}}
    raw_string: The raw string used to construct the MmcifObject.
  """
  file_id: str
  header: PdbHeader
  structure: PdbStructure
  chain_to_seqres: Mapping[ChainId, SeqRes]
  seqres_to_structure: Mapping[ChainId, Mapping[int, ResidueAtPosition]]
  raw_string: Any


@dataclasses.dataclass(frozen=True)
class ParsingResult:
  """Returned by the parse function.

  Contains:
    mmcif_object: A MmcifObject, may be None if no chain could be successfully
      parsed.
    errors: A dict mapping (file_id, chain_id) to any exception generated.
  """
  mmcif_object: Optional[MmcifObject]
  errors: Mapping[Tuple[str, str], Any]


class ParseError(Exception):
  """An error indicating that an mmCIF file could not be parsed."""


def mmcif_loop_to_list(prefix: str,
                       parsed_info: MmCIFDict) -> Sequence[Mapping[str, str]]:
  """Extracts loop associated with a prefix from mmCIF data as a list.

  Reference for loop_ in mmCIF:
    http://mmcif.wwpdb.org/docs/tutorials/mechanics/pdbx-mmcif-syntax.html

  Args:
    prefix: Prefix shared by each of the data items in the loop.
      e.g. '_entity_poly_seq.', where the data items are _entity_poly_seq.num,
      _entity_poly_seq.mon_id. Should include the trailing period.
    parsed_info: A dict of parsed mmCIF data, e.g. _mmcif_dict from a Biopython
      parser.

  Returns:
    Returns a list of dicts; each dict represents 1 entry from an mmCIF loop.
  """
  cols = []
  data = []
  for key, value in parsed_info.items():
    if key.startswith(prefix):
      cols.append(key)
      data.append(value)

  assert all([len(xs) == len(data[0]) for xs in data]), (
      'mmCIF error: Not all loops are the same length: %s' % cols)

  return [dict(zip(cols, xs)) for xs in zip(*data)]


def mmcif_loop_to_dict(prefix: str,
                       index: str,
                       parsed_info: MmCIFDict,
                       ) -> Mapping[str, Mapping[str, str]]:
  """Extracts loop associated with a prefix from mmCIF data as a dictionary.

  Args:
    prefix: Prefix shared by each of the data items in the loop.
      e.g. '_entity_poly_seq.', where the data items are _entity_poly_seq.num,
      _entity_poly_seq.mon_id. Should include the trailing period.
    index: Which item of loop data should serve as the key.
    parsed_info: A dict of parsed mmCIF data, e.g. _mmcif_dict from a Biopython
      parser.

  Returns:
    Returns a dict of dicts; each dict represents 1 entry from an mmCIF loop,
    indexed by the index column.
  """
  entries = mmcif_loop_to_list(prefix, parsed_info)
  return {entry[index]: entry for entry in entries}


@functools.lru_cache(16, typed=False)
def parse(*,
          file_id: str,
          mmcif_string: str,
          catch_all_errors: bool = True) -> ParsingResult:
  """Entry point, parses an mmcif_string.

  Args:
    file_id: A string identifier for this file. Should be unique within the
      collection of files being processed.
    mmcif_string: Contents of an mmCIF file.
    catch_all_errors: If True, all exceptions are caught and error messages are
      returned as part of the ParsingResult. If False exceptions will be allowed
      to propagate.

  Returns:
    A ParsingResult.
  """
  errors = {}
  try:
    parser = PDB.MMCIFParser(QUIET=True)
    handle = io.StringIO(mmcif_string)
    full_structure = parser.get_structure('', handle)
    first_model_structure = _get_first_model(full_structure)
    # Extract the _mmcif_dict from the parser, which contains useful fields not
    # reflected in the Biopython structure.
    parsed_info = parser._mmcif_dict  # pylint:disable=protected-access

    # Ensure all values are lists, even if singletons.
    for key, value in parsed_info.items():
      if not isinstance(value, list):
        parsed_info[key] = [value]

    header = _get_header(parsed_info)

    # Determine the protein chains, and their start numbers according to the
    # internal mmCIF numbering scheme (likely but not guaranteed to be 1).
    valid_chains = _get_protein_chains(parsed_info=parsed_info)
    if not valid_chains:
      return ParsingResult(
          None, {(file_id, ''): 'No protein chains found in this file.'})
    seq_start_num = {chain_id: min([monomer.num for monomer in seq])
                     for chain_id, seq in valid_chains.items()}

    # Loop over the atoms for which we have coordinates. Populate two mappings:
    # -mmcif_to_author_chain_id (maps internal mmCIF chain ids to chain ids used
    # the authors / Biopython).
    # -seq_to_structure_mappings (maps idx into sequence to ResidueAtPosition).
    mmcif_to_author_chain_id = {}
    seq_to_structure_mappings = {}
    for atom in _get_atom_site_list(parsed_info):
      if atom.model_num != '1':
        # We only process the first model at the moment.
        continue

      mmcif_to_author_chain_id[atom.mmcif_chain_id] = atom.author_chain_id

      if atom.mmcif_chain_id in valid_chains:
        hetflag = ' '
        if atom.hetatm_atom == 'HETATM':
          # Water atoms are assigned a special hetflag of W in Biopython. We
          # need to do the same, so that this hetflag can be used to fetch
          # a residue from the Biopython structure by id.
          if atom.residue_name in ('HOH', 'WAT'):
            hetflag = 'W'
          else:
            hetflag = 'H_' + atom.residue_name
        insertion_code = atom.insertion_code
        if not _is_set(atom.insertion_code):
          insertion_code = ' '
        position = ResiduePosition(chain_id=atom.author_chain_id,
                                   residue_number=int(atom.author_seq_num),
                                   insertion_code=insertion_code)
        seq_idx = int(atom.mmcif_seq_num) - seq_start_num[atom.mmcif_chain_id]
        current = seq_to_structure_mappings.get(atom.author_chain_id, {})
        current[seq_idx] = ResidueAtPosition(position=position,
                                             name=atom.residue_name,
                                             is_missing=False,
                                             hetflag=hetflag)
        seq_to_structure_mappings[atom.author_chain_id] = current

    # Add missing residue information to seq_to_structure_mappings.
    for chain_id, seq_info in valid_chains.items():
      author_chain = mmcif_to_author_chain_id[chain_id]
      current_mapping = seq_to_structure_mappings[author_chain]
      for idx, monomer in enumerate(seq_info):
        if idx not in current_mapping:
          current_mapping[idx] = ResidueAtPosition(position=None,
                                                   name=monomer.id,
                                                   is_missing=True,
                                                   hetflag=' ')

    author_chain_to_sequence = {}
    for chain_id, seq_info in valid_chains.items():
      author_chain = mmcif_to_author_chain_id[chain_id]
      seq = []
      for monomer in seq_info: 
        code = Bio.Data.IUPACData.protein_letters_3to1.get(monomer.id, 'X')
        seq.append(code if len(code) == 1 else 'X')
      seq = ''.join(seq)
      author_chain_to_sequence[author_chain] = seq

    mmcif_object = MmcifObject(
        file_id=file_id,
        header=header,
        structure=first_model_structure,
        chain_to_seqres=author_chain_to_sequence,
        seqres_to_structure=seq_to_structure_mappings,
        raw_string=parsed_info)

    return ParsingResult(mmcif_object=mmcif_object, errors=errors)
  except Exception as e:  # pylint:disable=broad-except
    errors[(file_id, '')] = e
    if not catch_all_errors:
      raise
    return ParsingResult(mmcif_object=None, errors=errors)


def _get_first_model(structure: PdbStructure) -> PdbStructure:
  """Returns the first model in a Biopython structure."""
  return next(structure.get_models())

_MIN_LENGTH_OF_CHAIN_TO_BE_COUNTED_AS_PEPTIDE = 21


def get_release_date(parsed_info: MmCIFDict) -> str:
  """Returns the oldest revision date."""
  revision_dates = parsed_info['_pdbx_audit_revision_history.revision_date']
  return min(revision_dates)


def _get_header(parsed_info: MmCIFDict) -> PdbHeader:
  """Returns a basic header containing method, release date and resolution."""
  header = {}

  experiments = mmcif_loop_to_list('_exptl.', parsed_info)
  header['structure_method'] = ','.join([
      experiment['_exptl.method'].lower() for experiment in experiments])

  # Note: The release_date here corresponds to the oldest revision. We prefer to
  # use this for dataset filtering over the deposition_date.
  if '_pdbx_audit_revision_history.revision_date' in parsed_info:
    header['release_date'] = get_release_date(parsed_info)
  else:
    logging.warning('Could not determine release_date: %s',
                    parsed_info['_entry.id'])

  header['resolution'] = 0.00
  for res_key in ('_refine.ls_d_res_high', '_em_3d_reconstruction.resolution',
                  '_reflns.d_resolution_high'):
    if res_key in parsed_info:
      try:
        raw_resolution = parsed_info[res_key][0]
        header['resolution'] = float(raw_resolution)
      except ValueError:
        logging.debug('Invalid resolution format: %s', parsed_info[res_key])

  return header


def _get_atom_site_list(parsed_info: MmCIFDict) -> Sequence[AtomSite]:
  """Returns list of atom sites; contains data not present in the structure."""
  return [AtomSite(*site) for site in zip(  # pylint:disable=g-complex-comprehension
      parsed_info['_atom_site.label_comp_id'],
      parsed_info['_atom_site.auth_asym_id'],
      parsed_info['_atom_site.label_asym_id'],
      parsed_info['_atom_site.auth_seq_id'],
      parsed_info['_atom_site.label_seq_id'],
      parsed_info['_atom_site.pdbx_PDB_ins_code'],
      parsed_info['_atom_site.group_PDB'],
      parsed_info['_atom_site.pdbx_PDB_model_num'],
      )]


def _get_protein_chains(
    *, parsed_info: Mapping[str, Any]) -> Mapping[ChainId, Sequence[Monomer]]:
  """Extracts polymer information for protein chains only.

  Args:
    parsed_info: _mmcif_dict produced by the Biopython parser.

  Returns:
    A dict mapping mmcif chain id to a list of Monomers.
  """
  # Get polymer information for each entity in the structure.
  entity_poly_seqs = mmcif_loop_to_list('_entity_poly_seq.', parsed_info)

  polymers = collections.defaultdict(list)
  for entity_poly_seq in entity_poly_seqs:
    polymers[entity_poly_seq['_entity_poly_seq.entity_id']].append(
        Monomer(id=entity_poly_seq['_entity_poly_seq.mon_id'],
                num=int(entity_poly_seq['_entity_poly_seq.num'])))

  # Get chemical compositions. Will allow us to identify which of these polymers
  # are proteins.
  chem_comps = mmcif_loop_to_dict('_chem_comp.', '_chem_comp.id', parsed_info)

  # Get chains information for each entity. Necessary so that we can return a
  # dict keyed on chain id rather than entity.
  struct_asyms = mmcif_loop_to_list('_struct_asym.', parsed_info)

  entity_to_mmcif_chains = collections.defaultdict(list)
  for struct_asym in struct_asyms:
    chain_id = struct_asym['_struct_asym.id']
    entity_id = struct_asym['_struct_asym.entity_id']
    entity_to_mmcif_chains[entity_id].append(chain_id)

  # Identify and return the valid protein chains.
  valid_chains = {}
  for entity_id, seq_info in polymers.items():
    chain_ids = entity_to_mmcif_chains[entity_id]

    # Reject polymers without any peptide-like components, such as DNA/RNA.
    if any(['peptide' in chem_comps[monomer.id]['_chem_comp.type'].lower()
            for monomer in seq_info]):
      for chain_id in chain_ids:
        valid_chains[chain_id] = seq_info
  return valid_chains


def _is_set(data: str) -> bool:
  """Returns False if data is a special mmCIF character indicating 'unset'."""
  return data not in ('.', '?')


In [73]:
hit_pdb_code = "4xov"
cif_string = open("/home/sirius/PhD/database/pdb_mmcif/mmcif_files/4xov.cif").read()
parsing_result = parse(file_id=hit_pdb_code, mmcif_string=cif_string)

In [77]:
parsing_result.mmcif_object.header

{'structure_method': 'x-ray diffraction',
 'release_date': '2015-09-09',
 'resolution': 1.2}

In [78]:
parsing_result.mmcif_object.file_id

'4xov'

In [85]:
parsing_result.mmcif_object.structure

<Model id=0>

### 2024 10. 15
alphafold differences in three models

In [3]:
import numpy as np
model_1_ptm_feature = {'aatype': (4, 468), 'residue_index': (4, 468), 'seq_length': (4,), 'template_aatype': (4, 4, 468), 'template_all_atom_masks': (4, 4, 468, 37), 'template_all_atom_positions': (4, 4, 468, 37, 3), 'template_sum_probs': (4, 4, 1), 'is_distillation': (4,), 'seq_mask': (4, 468), 'msa_mask': (4, 508, 468), 'msa_row_mask': (4, 508), 'random_crop_to_size_seed': (4, 2), 'template_mask': (4, 4), 'template_pseudo_beta': (4, 4, 468, 3), 'template_pseudo_beta_mask': (4, 4, 468), 'atom14_atom_exists': (4, 468, 14), 'residx_atom14_to_atom37': (4, 468, 14), 'residx_atom37_to_atom14': (4, 468, 37), 'atom37_atom_exists': (4, 468, 37), 'extra_msa': (4, 5120, 468), 'extra_msa_mask': (4, 5120, 468), 'extra_msa_row_mask': (4, 5120), 'bert_mask': (4, 508, 468), 'true_msa': (4, 508, 468), 'extra_has_deletion': (4, 5120, 468), 'extra_deletion_value': (4, 5120, 468), 'msa_feat': (4, 508, 468, 49), 'target_feat': (4, 468, 22)}
model_2_ptm_feature = {'aatype': (4, 468), 'residue_index': (4, 468), 'seq_length': (4,), 'template_aatype': (4, 4, 468), 'template_all_atom_masks': (4, 4, 468, 37), 'template_all_atom_positions': (4, 4, 468, 37, 3), 'template_sum_probs': (4, 4, 1), 'is_distillation': (4,), 'seq_mask': (4, 468), 'msa_mask': (4, 508, 468), 'msa_row_mask': (4, 508), 'random_crop_to_size_seed': (4, 2), 'template_mask': (4, 4), 'template_pseudo_beta': (4, 4, 468, 3), 'template_pseudo_beta_mask': (4, 4, 468), 'atom14_atom_exists': (4, 468, 14), 'residx_atom14_to_atom37': (4, 468, 14), 'residx_atom37_to_atom14': (4, 468, 37), 'atom37_atom_exists': (4, 468, 37), 'extra_msa': (4, 1024, 468), 'extra_msa_mask': (4, 1024, 468), 'extra_msa_row_mask': (4, 1024), 'bert_mask': (4, 508, 468), 'true_msa': (4, 508, 468), 'extra_has_deletion': (4, 1024, 468), 'extra_deletion_value': (4, 1024, 468), 'msa_feat': (4, 508, 468, 49), 'target_feat': (4, 468, 22)}
model_3_ptm_feature = {'aatype': (4, 468), 'residue_index': (4, 468), 'seq_length': (4,), 'is_distillation': (4,), 'seq_mask': (4, 468), 'msa_mask': (4, 512, 468), 'msa_row_mask': (4, 512), 'random_crop_to_size_seed': (4, 2), 'atom14_atom_exists': (4, 468, 14), 'residx_atom14_to_atom37': (4, 468, 14), 'residx_atom37_to_atom14': (4, 468, 37), 'atom37_atom_exists': (4, 468, 37), 'extra_msa': (4, 5120, 468), 'extra_msa_mask': (4, 5120, 468), 'extra_msa_row_mask': (4, 5120), 'bert_mask': (4, 512, 468), 'true_msa': (4, 512, 468), 'extra_has_deletion': (4, 5120, 468), 'extra_deletion_value': (4, 5120, 468), 'msa_feat': (4, 512, 468, 49), 'target_feat': (4, 468, 22)}
model_4_ptm_feature = {'aatype': (4, 468), 'residue_index': (4, 468), 'seq_length': (4,), 'is_distillation': (4,), 'seq_mask': (4, 468), 'msa_mask': (4, 512, 468), 'msa_row_mask': (4, 512), 'random_crop_to_size_seed': (4, 2), 'atom14_atom_exists': (4, 468, 14), 'residx_atom14_to_atom37': (4, 468, 14), 'residx_atom37_to_atom14': (4, 468, 37), 'atom37_atom_exists': (4, 468, 37), 'extra_msa': (4, 5120, 468), 'extra_msa_mask': (4, 5120, 468), 'extra_msa_row_mask': (4, 5120), 'bert_mask': (4, 512, 468), 'true_msa': (4, 512, 468), 'extra_has_deletion': (4, 5120, 468), 'extra_deletion_value': (4, 5120, 468), 'msa_feat': (4, 512, 468, 49), 'target_feat': (4, 468, 22)}
model_5_ptm_feature = {'aatype': (4, 468), 'residue_index': (4, 468), 'seq_length': (4,), 'is_distillation': (4,), 'seq_mask': (4, 468), 'msa_mask': (4, 512, 468), 'msa_row_mask': (4, 512), 'random_crop_to_size_seed': (4, 2), 'atom14_atom_exists': (4, 468, 14), 'residx_atom14_to_atom37': (4, 468, 14), 'residx_atom37_to_atom14': (4, 468, 37), 'atom37_atom_exists': (4, 468, 37), 'extra_msa': (4, 1024, 468), 'extra_msa_mask': (4, 1024, 468), 'extra_msa_row_mask': (4, 1024), 'bert_mask': (4, 512, 468), 'true_msa': (4, 512, 468), 'extra_has_deletion': (4, 1024, 468), 'extra_deletion_value': (4, 1024, 468), 'msa_feat': (4, 512, 468, 49), 'target_feat': (4, 468, 22)}

In [12]:
for key in model_5_ptm_feature.keys():
    print(key,model_5_ptm_feature[key])

aatype (4, 468)
residue_index (4, 468)
seq_length (4,)
is_distillation (4,)
seq_mask (4, 468)
msa_mask (4, 512, 468)
msa_row_mask (4, 512)
random_crop_to_size_seed (4, 2)
atom14_atom_exists (4, 468, 14)
residx_atom14_to_atom37 (4, 468, 14)
residx_atom37_to_atom14 (4, 468, 37)
atom37_atom_exists (4, 468, 37)
extra_msa (4, 1024, 468)
extra_msa_mask (4, 1024, 468)
extra_msa_row_mask (4, 1024)
bert_mask (4, 512, 468)
true_msa (4, 512, 468)
extra_has_deletion (4, 1024, 468)
extra_deletion_value (4, 1024, 468)
msa_feat (4, 512, 468, 49)
target_feat (4, 468, 22)


In [13]:
for key in model_1_ptm_feature.keys():
    print(key,model_1_ptm_feature[key])

aatype (4, 468)
residue_index (4, 468)
seq_length (4,)
template_aatype (4, 4, 468)
template_all_atom_masks (4, 4, 468, 37)
template_all_atom_positions (4, 4, 468, 37, 3)
template_sum_probs (4, 4, 1)
is_distillation (4,)
seq_mask (4, 468)
msa_mask (4, 508, 468)
msa_row_mask (4, 508)
random_crop_to_size_seed (4, 2)
template_mask (4, 4)
template_pseudo_beta (4, 4, 468, 3)
template_pseudo_beta_mask (4, 4, 468)
atom14_atom_exists (4, 468, 14)
residx_atom14_to_atom37 (4, 468, 14)
residx_atom37_to_atom14 (4, 468, 37)
atom37_atom_exists (4, 468, 37)
extra_msa (4, 5120, 468)
extra_msa_mask (4, 5120, 468)
extra_msa_row_mask (4, 5120)
bert_mask (4, 508, 468)
true_msa (4, 508, 468)
extra_has_deletion (4, 5120, 468)
extra_deletion_value (4, 5120, 468)
msa_feat (4, 508, 468, 49)
target_feat (4, 468, 22)


In [5]:
feature = np.load("/home/sirius/PhD/software/alphafold/output_without_MSA/xcf_mc9bp81/features.pkl",allow_pickle=True)

In [7]:
feature.keys()

dict_keys(['aatype', 'between_segment_residues', 'domain_name', 'residue_index', 'seq_length', 'sequence', 'deletion_matrix_int', 'msa', 'num_alignments', 'msa_species_identifiers', 'template_aatype', 'template_all_atom_masks', 'template_all_atom_positions', 'template_domain_names', 'template_sequence', 'template_sum_probs'])

In [8]:
for i in feature.keys():
    print(i,feature[i].shape)

aatype (468, 21)
between_segment_residues (468,)
domain_name (1,)
residue_index (468,)
seq_length (468,)
sequence (1,)
deletion_matrix_int (1, 468)
msa (1, 468)
num_alignments (468,)
msa_species_identifiers (1,)
template_aatype (0,)
template_all_atom_masks (0,)
template_all_atom_positions (0,)
template_domain_names (0,)
template_sequence (0,)
template_sum_probs (0,)
